In [2]:
%load_ext autoreload
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
from dbArtistsGenius import dbArtistsGenius
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial


######################################################
# Juypter
######################################################
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))



################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
dbAP = dbArtistsDeezer()
retval = dbAP.artist.getData('/Volumes/Piggy/Discog/artists-deezer/2/89128202.p')
retval.show()

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
dbAP = dbArtistsDeezer()
dbAP.parse(2, previousDays=1, force=True)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer

for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDeezer())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [ ]:
import urllib
def downloadURL(url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    print("Now Downloading {0}".format(url))

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need
        
    return data, response.getcode()

#data, code = downloadURL("https://www.deezer.com/us/artist/5395257/top_track")

In [ ]:
url="https://www.iheart.com/artist/helgi-bjornsson-32568452/albums"
data, code = downloadURL(url)
from ioUtils import saveFile, getFile
saveFile(idata=data, ifile="iheart_albums.p")
from time import sleep
sleep(3)
url="https://www.iheart.com/artist/helgi-bjornsson-32568452/songs"
data, code = downloadURL(url)
from ioUtils import saveFile, getFile
saveFile(idata=data, ifile="iheart_songs.p")

In [ ]:
from webUtils import getHTML
bsdata = getHTML("iheart_test.p")

In [9]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDeezer())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   deezer   =========================
  ===> Parsing all 66 files for modval 0
  ===> Forcing Reloads of ModVal=0
	Processed 0/66 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 66 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/0-DB.p
Saving 66 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/0-DB.p
Saving 1499 total artist media
  --> This file is 65.9kB.
  ===> Found 66 previous data for ModVal=0
Saving 66 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/0-Metadata.p
  --> This file is 2.4kB.
  ===> Found 66 previous data for ModVal=0
Saving 66 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/0-MediaMetadata.p
  --> This file is 3.6kB.
=========================   deezer   ============

No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 62 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/8-DB.p
Saving 62 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/8-DB.p
Saving 1257 total artist media
  --> This file is 56.2kB.
  ===> Found 62 previous data for ModVal=8
Saving 62 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/8-Metadata.p
  --> This file is 2.2kB.
  ===> Found 62 previous data for ModVal=8
Saving 62 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/8-MediaMetadata.p
  --> This file is 3.2kB.
=========================   deezer   =========================
  ===> Parsing all 58 files for modval 9
  ===> Forcing Reloads of ModVal=9
	Processed 0/58 files.
No track table
No track table
No track table
No track table
No track table
No track table
No

No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 81 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/16-DB.p
Saving 81 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/16-DB.p
Saving 1656 total artist media
  --> This file is 73.8kB.
  ===> Found 81 previous data for ModVal=16
Saving 81 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/16-Metadata.p
  --> This file is 2.9kB.
  ===> Found 81 previous data for ModVal=16
Saving 81 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/16-MediaMetadata.p
  --> This file is 4.3kB.
=========================   deezer   =========================
  ===> Parsing all 63 files for modval 17
  ===> Forcing Reloads of ModVal=17
	Processed 0/63 files.
No track table
No track table
No track 

No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 77 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/24-DB.p
Saving 77 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/24-DB.p
Saving 1860 total artist media
  --> This file is 79.6kB.
  ===> Found 77 previous data for ModVal=24
Saving 77 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/24-Metadata.p
  --> This file is 2.9kB.
  ===> Found 77 previous data for ModVal=24
Saving 77 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/24-MediaMetadata.p
  --> This file is 3.8kB.
=========================   deezer   =========================
  ===> Parsing all 65 files for modval 25
  ===> Forcing Reloads of ModVal=25
	Processed 0/65 files.
No track 

No track table
No track table
No track table
Saving 165 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/32-DB.p
Saving 165 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/32-DB.p
Saving 2166 total artist media
  --> This file is 107.4kB.
  ===> Found 165 previous data for ModVal=32
Saving 165 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/32-Metadata.p
  --> This file is 6.3kB.
  ===> Found 165 previous data for ModVal=32
Saving 165 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/32-MediaMetadata.p
  --> This file is 7.8kB.
=========================   deezer   =========================
  ===> Parsing all 70 files for modval 33
  ===> Forcing Reloads of ModVal=33
	Processed 0/70 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 70 new art

...

  --> This file is 58.9kB.
  ===> Found 72 previous data for ModVal=65
Saving 72 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/65-Metadata.p
  --> This file is 2.6kB.
  ===> Found 72 previous data for ModVal=65
Saving 72 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/65-MediaMetadata.p
  --> This file is 3.7kB.
=========================   deezer   =========================
  ===> Parsing all 69 files for modval 66
  ===> Forcing Reloads of ModVal=66
	Processed 0/69 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 69 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/66-DB.p
Saving 69 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/66-DB.p
Saving 1489 total artist media
  --> This file is 66.0kB.
  ===> Found 69 previous data for ModVal=66
Saving 69 new artist IDs name data to /User

  ===> Parsing all 65 files for modval 74
  ===> Forcing Reloads of ModVal=74
	Processed 0/65 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 65 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/74-DB.p
Saving 65 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/74-DB.p
Saving 1416 total artist media
  --> This file is 63.7kB.
  ===> Found 65 previous data for ModVal=74
Saving 65 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/74-Metadata.p
  --> This file is 2.4kB.
  ===> Found 65 previous data for ModVal=74
Saving 65 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/74-MediaMetadata.p
  --> This file is 3.4kB.
=========================   deezer   =========================
  ===> Parsing all 68 files for modval 75
  ===> Forcing Reloads of ModVal=75
	Proce

No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 178 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/82-DB.p
Saving 178 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/82-DB.p
Saving 2058 total artist media
  --> This file is 105.2kB.
  ===> Found 178 previous data for ModVal=82
Saving 178 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/82-Metadata.p
  --> This file is 6.9kB.
  ===> Found 178 previous data for ModVal=82
Saving 178 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/82-MediaMetadata.p
  --> This file is 8.2kB.
=========================   dee

  ===> Parsing all 99 files for modval 91
  ===> Forcing Reloads of ModVal=91
	Processed 0/99 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 99 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/91-DB.p
Saving 99 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/91-DB.p
Saving 2021 total artist media
  --> This file is 91.2kB.
  ===> Found 99 previous data for ModVal=91
Saving 99 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/91-Metadata.p
  --> This file is 3.5kB.
  ===> Found 99 previous data for ModVal=91
Saving 99 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/91-MediaMetadata.p
  --> This file is 5.1kB.
=========================  

No track table
No track table
No track table
No track table
Saving 69 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/99-DB.p
Saving 69 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/99-DB.p
Saving 1490 total artist media
  --> This file is 67.3kB.
  ===> Found 69 previous data for ModVal=99
Saving 69 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/99-Metadata.p
  --> This file is 2.5kB.
  ===> Found 69 previous data for ModVal=99
Saving 69 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/99-MediaMetadata.p
  --> This file is 3.6kB.


In [8]:
%load_ext autoreload
%autoreload
from dbArtistsGenius import dbArtistsGenius
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsGenius())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   genius   =========================
  ===> Parsing all 3 files for modval 0
  ===> Forcing Reloads of ModVal=0
	Processed 0/3 files.
Saving 3 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/0-DB.p
Saving 3 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/0-DB.p
Saving 43 total artist media
  --> This file is 2.2kB.
  ===> Found 3 previous data for ModVal=0
Saving 3 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/0-Metadata.p
  --> This file is 177B.
  ===> Found 3 previous data for ModVal=0
Saving 3 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/0-MediaMetadata.p
  --> This file is 962B.
=========================   genius   =========================
  ===> Parsing all 0 files for modval 1
  ===> Forcing Reloads of ModVal=1
=========================   genius   =======

Saving 3 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/13-DB.p
Saving 3 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/13-DB.p
Saving 50 total artist media
  --> This file is 2.4kB.
  ===> Found 3 previous data for ModVal=13
Saving 3 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/13-Metadata.p
  --> This file is 192B.
  ===> Found 3 previous data for ModVal=13
Saving 3 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/13-MediaMetadata.p
  --> This file is 1.2kB.
=========================   genius   =========================
  ===> Parsing all 2 files for modval 14
  ===> Forcing Reloads of ModVal=14
	Processed 0/2 files.
Saving 2 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/14-DB.p
Saving 2 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/14-DB.p
Saving 15 total artist media
  --> This file is 1.2kB.
  ===> Found 2 previous data for ModVal=

Saving 2 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/28-DB.p
Saving 2 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/28-DB.p
Saving 22 total artist media
  --> This file is 1.4kB.
  ===> Found 2 previous data for ModVal=28
Saving 2 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/28-Metadata.p
  --> This file is 140B.
  ===> Found 2 previous data for ModVal=28
Saving 2 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/28-MediaMetadata.p
  --> This file is 589B.
=========================   genius   =========================
  ===> Parsing all 0 files for modval 29
  ===> Forcing Reloads of ModVal=29
=========================   genius   =========================
  ===> Parsing all 3 files for modval 30
  ===> Forcing Reloads of ModVal=30
	Processed 0/3 files.
Saving 3 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/30-DB.p
Saving 3 total artist IDs to /Users/tgad

Saving 4 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/45-DB.p
Saving 4 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/45-DB.p
Saving 38 total artist media
  --> This file is 2.3kB.
  ===> Found 4 previous data for ModVal=45
Saving 4 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/45-Metadata.p
  --> This file is 234B.
  ===> Found 4 previous data for ModVal=45
Saving 4 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/45-MediaMetadata.p
  --> This file is 978B.
=========================   genius   =========================
  ===> Parsing all 3 files for modval 46
  ===> Forcing Reloads of ModVal=46
	Processed 0/3 files.
Saving 3 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/46-DB.p
Saving 3 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/46-DB.p
Saving 46 total artist media
  --> This file is 2.3kB.
  ===> Found 3 previous data for ModVal=4

Saving 5 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/62-DB.p
Saving 5 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/62-DB.p
Saving 33 total artist media
  --> This file is 2.2kB.
  ===> Found 5 previous data for ModVal=62
Saving 5 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/62-Metadata.p
  --> This file is 278B.
  ===> Found 5 previous data for ModVal=62
Saving 5 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/62-MediaMetadata.p
  --> This file is 863B.
=========================   genius   =========================
  ===> Parsing all 3 files for modval 63
  ===> Forcing Reloads of ModVal=63
	Processed 0/3 files.
Saving 3 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/63-DB.p
Saving 3 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/63-DB.p
Saving 32 total artist media
  --> This file is 1.9kB.
  ===> Found 3 previous data for ModVal=6

Saving 3 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/76-DB.p
Saving 3 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/76-DB.p
Saving 36 total artist media
  --> This file is 2.0kB.
  ===> Found 3 previous data for ModVal=76
Saving 3 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/76-Metadata.p
  --> This file is 166B.
  ===> Found 3 previous data for ModVal=76
Saving 3 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/76-MediaMetadata.p
  --> This file is 916B.
=========================   genius   =========================
  ===> Parsing all 2 files for modval 77
  ===> Forcing Reloads of ModVal=77
	Processed 0/2 files.
Saving 2 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/77-DB.p
Saving 2 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/77-DB.p
Saving 13 total artist media
  --> This file is 1.2kB.
  ===> Found 2 previous data for ModVal=7

Saving 2 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/90-DB.p
Saving 2 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/90-DB.p
Saving 23 total artist media
  --> This file is 1.5kB.
  ===> Found 2 previous data for ModVal=90
Saving 2 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/90-Metadata.p
  --> This file is 147B.
  ===> Found 2 previous data for ModVal=90
Saving 2 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-genius-db/metadata/90-MediaMetadata.p
  --> This file is 599B.
=========================   genius   =========================
  ===> Parsing all 1 files for modval 91
  ===> Forcing Reloads of ModVal=91
	Processed 0/1 files.
Saving 1 new artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/91-DB.p
Saving 1 total artist IDs to /Users/tgadfort/Music/Discog/artists-genius-db/91-DB.p
Saving 13 total artist media
  --> This file is 1.0kB.
  ===> Found 1 previous data for ModVal=9

In [6]:
%load_ext autoreload
%autoreload
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertCredit, dbArtistsRawHTML

dbAP = dbArtistsRawHTML(dbArtistsRateYourMusic())
dbAP.parse(previousDays=0.1)
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsRateYourMusic())
    dometa = dbAP.parse(modVal, previousDays=0.1, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   rateyourmusic   =========================
  ===> Found new 29 files (< 0.1 days) to parse
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/DISH:: Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 25.8kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Sari Osawa Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 18.0kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/JóiPé Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 19.0kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Aron Can Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 19.9kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Herra Hnetusmjör Albums: songs, disco

Saving 1 new artist IDs to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/6-DB.p
Saving 17 total artist IDs to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/6-DB.p
Saving 984 total artist media
  --> This file is 86.3kB.
  ===> Found 17 previous data for ModVal=6
Saving 17 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/metadata/6-Metadata.p
  --> This file is 714B.
  ===> Found 17 previous data for ModVal=6
Saving 17 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/metadata/6-MediaMetadata.p
  --> This file is 49.8kB.
=========================   rateyourmusic   =========================
  ===> Found new 2 files (< 0.1 days) to parse for modval 7
  ===> Found 17 previous data for ModVal=7
Saving 2 new artist IDs to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/7-DB.p
Saving 17 total artist IDs to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/7-DB.p
Saving 802 total artist media
  --> This 

  --> This file is 43.5kB.
  ===> Found 9 previous data for ModVal=21
Saving 9 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/metadata/21-Metadata.p
  --> This file is 452B.
  ===> Found 9 previous data for ModVal=21
Saving 9 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/metadata/21-MediaMetadata.p
  --> This file is 24.9kB.
=========================   rateyourmusic   =========================
  ===> Found new 0 files (< 0.1 days) to parse for modval 22
  ===> Found 16 previous data for ModVal=22
=========================   rateyourmusic   =========================
  ===> Found new 0 files (< 0.1 days) to parse for modval 23
  ===> Found 19 previous data for ModVal=23
=========================   rateyourmusic   =========================
  ===> Found new 1 files (< 0.1 days) to parse for modval 24
  ===> Found 11 previous data for ModVal=24
Saving 1 new artist IDs to /Users/tgadfort/Music/Discog/artists-rateyourmusic

  --> This file is 87.2kB.
  ===> Found 24 previous data for ModVal=42
Saving 24 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/metadata/42-Metadata.p
  --> This file is 997B.
  ===> Found 24 previous data for ModVal=42
Saving 24 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/metadata/42-MediaMetadata.p
  --> This file is 50.1kB.
=========================   rateyourmusic   =========================
  ===> Found new 1 files (< 0.1 days) to parse for modval 43
  ===> Found 21 previous data for ModVal=43
Saving 1 new artist IDs to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/43-DB.p
Saving 21 total artist IDs to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/43-DB.p
Saving 997 total artist media
  --> This file is 85.6kB.
  ===> Found 21 previous data for ModVal=43
Saving 21 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/metadata/43-Metadata.p
  --> This file is 875B.
 

  --> This file is 40.5kB.
  ===> Found 16 previous data for ModVal=61
Saving 16 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/metadata/61-Metadata.p
  --> This file is 754B.
  ===> Found 16 previous data for ModVal=61
Saving 16 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/metadata/61-MediaMetadata.p
  --> This file is 23.4kB.
=========================   rateyourmusic   =========================
  ===> Found new 0 files (< 0.1 days) to parse for modval 62
  ===> Found 13 previous data for ModVal=62
=========================   rateyourmusic   =========================
  ===> Found new 0 files (< 0.1 days) to parse for modval 63
  ===> Found 12 previous data for ModVal=63
=========================   rateyourmusic   =========================
  ===> Found new 0 files (< 0.1 days) to parse for modval 64
  ===> Found 11 previous data for ModVal=64
=========================   rateyourmusic   =========================
  =

  ===> Found 14 previous data for ModVal=80
=========================   rateyourmusic   =========================
  ===> Found new 0 files (< 0.1 days) to parse for modval 81
  ===> Found 13 previous data for ModVal=81
=========================   rateyourmusic   =========================
  ===> Found new 1 files (< 0.1 days) to parse for modval 82
  ===> Found 18 previous data for ModVal=82
Saving 1 new artist IDs to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/82-DB.p
Saving 18 total artist IDs to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/82-DB.p
Saving 650 total artist media
  --> This file is 56.9kB.
  ===> Found 18 previous data for ModVal=82
Saving 18 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/metadata/82-Metadata.p
  --> This file is 768B.
  ===> Found 18 previous data for ModVal=82
Saving 18 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-rateyourmusic-db/metadata/82-MediaMetadata.p
  --> This file is 3

In [11]:
from ioUtils import getFile
data = getFile("/Users/tgadfort/Music/Discog/db-rateyourmusic/ArtistIDToName.p")

In [18]:
from pandas import DataFrame, Series
for irow,row in DataFrame(Series(data)).iterrows():
    print(irow,row)

188906 0    Adolph Green
Name: 188906, dtype: object
406 0    Backstreet Boys
Name: 406, dtype: object
6 0    Pulp
Name: 6, dtype: object
13306 0    Majestic
Name: 13306, dtype: object
21506 0    Blue Magic
Name: 21506, dtype: object
4606 0    E-40
Name: 4606, dtype: object
3306 0    Heather Nova
Name: 3306, dtype: object
206 0    The Human League
Name: 206, dtype: object
7506 0    The Toasters
Name: 7506, dtype: object
4805511706 0    Santana
Name: 4805511706, dtype: object
6371695506 0    The Libertines
Name: 6371695506, dtype: object
5629667006 0    The Pop Gun
Name: 5629667006, dtype: object
658285006 0    Alan Jackson
Name: 658285006, dtype: object
579036406 0    10cc
Name: 579036406, dtype: object
4909028406 0    Whitesnake
Name: 4909028406, dtype: object
5258932206 0    The Phil Collins Big Band
Name: 5258932206, dtype: object
2754112806 0    Tove Lo
Name: 2754112806, dtype: object
39294 0    112
Name: 39294, dtype: object
2894 0    Adam Ant
Name: 2894, dtype: object
694 0    Ga

240580 0    Rodgers & Hammerstein
Name: 240580, dtype: object
980 0    Five for Fighting
Name: 980, dtype: object
6250489580 0    Rowan Brothers
Name: 6250489580, dtype: object
2240088780 0    Alison Krauss
Name: 2240088780, dtype: object
7661383180 0    Aaliyah
Name: 7661383180, dtype: object
9248308380 0    Nick Ashford
Name: 9248308380, dtype: object
8006489980 0    Young B
Name: 8006489980, dtype: object
3618833980 0    Apothys
Name: 3618833980, dtype: object
6493758280 0    John Tesh
Name: 6493758280, dtype: object
8600538380 0    "Weird Al" Yankovic
Name: 8600538380, dtype: object
811066080 0    Dua Lipa
Name: 811066080, dtype: object
2591 0    Alan Jackson
Name: 2591, dtype: object
65091 0    Daddy Yankee
Name: 65091, dtype: object
2491 0    Qango
Name: 2491, dtype: object
891 0    Peter Frampton
Name: 891, dtype: object
1591 0    Lit
Name: 1591, dtype: object
8691 0    Charlie Rich
Name: 8691, dtype: object
127791 0    Edsilia Rombley
Name: 127791, dtype: object
1748270991 0   

3420 0    Royce da 5'9"
Name: 3420, dtype: object
54620 0    The Beatniks
Name: 54620, dtype: object
5420 0    Within Temptation
Name: 5420, dtype: object
130920 0    Glennis Grace
Name: 130920, dtype: object
3307006520 0    Sevendust
Name: 3307006520, dtype: object
2435596420 0    Peter Banks
Name: 2435596420, dtype: object
8346287720 0    Echo & The Bunnymen
Name: 8346287720, dtype: object
1985170820 0    Betty Comden
Name: 1985170820, dtype: object
3253460120 0    Barry Manilow
Name: 3253460120, dtype: object
7675907320 0    Matthew Good Band
Name: 7675907320, dtype: object
9439609720 0    John Mayer Trio
Name: 9439609720, dtype: object
8722586920 0    The Zoo
Name: 8722586920, dtype: object
3845030120 0    Jack King
Name: 3845030120, dtype: object
2174792420 0    Mike Love
Name: 2174792420, dtype: object
9004659220 0    Chloe Agnew
Name: 9004659220, dtype: object
3437312020 0    Арсенал Arsenal
Name: 3437312020, dtype: object
9571280820 0    עידן עמדי Idan Amedi
Name: 9571280820, d

Name: 16564, dtype: object
2063273464 0    Bernie Williams
Name: 2063273464, dtype: object
7178441164 0    Royal Philharmonic Orchestra
Name: 7178441164, dtype: object
155293464 0    Øresund Space Collective
Name: 155293464, dtype: object
173753 0    PostMortem
Name: 173753, dtype: object
62353 0    Jaheim
Name: 62353, dtype: object
1101453 0    NF
Name: 1101453, dtype: object
14053 0    The Derek Trucks Band
Name: 14053, dtype: object
600453 0    Disclosure
Name: 600453, dtype: object
258253 0    Steve Aoki
Name: 258253, dtype: object
58053 0    Enzo Enzo
Name: 58053, dtype: object
441053 0    Kris Allen
Name: 441053, dtype: object
318929953 0    Jr. Boss
Name: 318929953, dtype: object
6313275953 0    Edwin Starr
Name: 6313275953, dtype: object
8232796953 0    The Oak Ridge Boys
Name: 8232796953, dtype: object
2254878553 0    Pallas
Name: 2254878553, dtype: object
4587789053 0    Glimmer
Name: 4587789053, dtype: object
491907753 0    Zé Neto & Cristiano
Name: 491907753, dtype: object


Name: 2160459693, dtype: object
6101 0    Obliveon
Name: 6101, dtype: object
280901 0    Pino Palladino
Name: 280901, dtype: object
6301 0    Four Tops
Name: 6301, dtype: object
8201 0    James Carr
Name: 8201, dtype: object
8972087801 0    Lana Lane
Name: 8972087801, dtype: object
4225664901 0    Lifehouse
Name: 4225664901, dtype: object
7009942901 0    Earth, Wind & Fire
Name: 7009942901, dtype: object
9304061301 0    Shutdown
Name: 9304061301, dtype: object
6987680801 0    Barbara Mandrell
Name: 6987680801, dtype: object
599239 0    Joe Jackson
Name: 599239, dtype: object
1539 0    Alphaville
Name: 1539, dtype: object
39 0    Paul Simon
Name: 39, dtype: object
2539 0    LL Cool J
Name: 2539, dtype: object
11039 0    Fred Wesley
Name: 11039, dtype: object
3239 0    Foghat
Name: 3239, dtype: object
6939 0    Blue Meanies
Name: 6939, dtype: object
4339 0    Barry White
Name: 4339, dtype: object
84739 0    The Rationals
Name: 84739, dtype: object
766489239 0    The Makers
Name: 76648923

1929091573 0    The Challengers
Name: 1929091573, dtype: object
9100524673 0    Angie Stone
Name: 9100524673, dtype: object
2304085773 0    Aloe Blacc
Name: 2304085773, dtype: object
5013976473 0    Bee Gees
Name: 5013976473, dtype: object
5224533173 0    Michael Jackson
Name: 5224533173, dtype: object
3555973273 0    Apothis
Name: 3555973273, dtype: object
8616930273 0    Birdman
Name: 8616930273, dtype: object
7428574573 0    Scuba
Name: 7428574573, dtype: object
4207752473 0    The Kingsway Symphony Orchestra
Name: 4207752473, dtype: object
14626 0    Sacred Reich
Name: 14626, dtype: object
626 0    10cc
Name: 626, dtype: object
252526 0    AKB48
Name: 252526, dtype: object
389526 0    Selena Gomez
Name: 389526, dtype: object
11426 0    Valerie Simpson
Name: 11426, dtype: object
191341026 0    Young Fame
Name: 191341026, dtype: object
4851940126 0    Kid Cudi
Name: 4851940126, dtype: object
796441126 0    Stephen Bishop
Name: 796441126, dtype: object
4185771226 0    Ariana Grande
Na

98 0    Dave Matthews Band
Name: 98, dtype: object
146198 0    Pure 13
Name: 146198, dtype: object
28998 0    New Glory
Name: 28998, dtype: object
2802633798 0    Mr. Big
Name: 2802633798, dtype: object
9916090198 0    Dr. Dre
Name: 9916090198, dtype: object
3275709698 0    The Human League
Name: 3275709698, dtype: object
1582255698 0    Murder Squad
Name: 1582255698, dtype: object
5724880398 0    The Beat
Name: 5724880398, dtype: object
1574363698 0    Steve Augeri
Name: 1574363698, dtype: object
3631643598 0    Sioux Falls
Name: 3631643598, dtype: object
2425236998 0    MC Dede
Name: 2425236998, dtype: object
2606242798 0    The Angels
Name: 2606242798, dtype: object
146419 0    Exploding Seagulls
Name: 146419, dtype: object
661419 0    Naturecore
Name: 661419, dtype: object
419 0    Suzanne Vega
Name: 419, dtype: object
6219 0    Johnny Mathis
Name: 6219, dtype: object
119 0    Lifehouse
Name: 119, dtype: object
54819 0    Abraxas Pool
Name: 54819, dtype: object
410269219 0    Frank

Name: 524, dtype: object
894924 0    J Balvin
Name: 894924, dtype: object
970224 0    Doja Cat
Name: 970224, dtype: object
2674167824 0    The Jicks
Name: 2674167824, dtype: object
7497308524 0    Stevie Ray Vaughan
Name: 7497308524, dtype: object
5979530824 0    Freddie Jackson
Name: 5979530824, dtype: object
328027224 0    Richard Rodgers
Name: 328027224, dtype: object
2954451124 0    T'Pau
Name: 2954451124, dtype: object
2412152224 0    Wigwam
Name: 2412152224, dtype: object
4338 0    "Weird Al" Yankovic
Name: 4338, dtype: object
1538 0    38 Special
Name: 1538, dtype: object
438 0    B.B. King
Name: 438, dtype: object
96638 0    The Steepwater Band
Name: 96638, dtype: object
644238 0    JD McPherson
Name: 644238, dtype: object
3038 0    Rakim
Name: 3038, dtype: object
5017233938 0    Warrant
Name: 5017233938, dtype: object
5432848638 0    The For Carnation
Name: 5432848638, dtype: object
4908178438 0    Fountains of Wayne
Name: 4908178438, dtype: object
5164222738 0    Double X
Nam

2394609697 0    No Use for a Name
Name: 2394609697, dtype: object
3798213597 0    Stormzy
Name: 3798213597, dtype: object
3904391197 0    Tommy Shaw
Name: 3904391197, dtype: object
9566865297 0    George Tickner
Name: 9566865297, dtype: object
2083210497 0    ゆず Yuzu
Name: 2083210497, dtype: object
15868 0    Sirenia
Name: 15868, dtype: object
8068 0    2 Unlimited
Name: 8068, dtype: object
2568 0    Gipsy Kings
Name: 2568, dtype: object
78691068 0    Joe Jackson
Name: 78691068, dtype: object
447746668 0    alt-J
Name: 447746668, dtype: object
7072894968 0    Monster Squad
Name: 7072894968, dtype: object
3952449568 0    Blazing Salads
Name: 3952449568, dtype: object
6955080068 0    Ricky Martin
Name: 6955080068, dtype: object
1948369468 0    Born in Exile
Name: 1948369468, dtype: object
6266580868 0    A.D.O.R.
Name: 6266580868, dtype: object
8415844568 0    Randy Jackson
Name: 8415844568, dtype: object
6970318468 0    Eri Esittäjiä
Name: 6970318468, dtype: object
8044474868 0    Винт 

In [25]:
%autoreload
from artistRateYourMusic import artistRateYourMusic
ar = artistRateYourMusic()
retval = ar.getData("/Volumes/Piggy/Discog/artists-rateyourmusic/22/1378079422.p")
retval.show()

Artist Data Class
-------------------------
Artist:  Golden Bomber (ゴールデンボンバー Golden Bomber)
Meta:    ゴールデンボンバー discography - RYM/Sonemic
         https://rateyourmusic.com/artist/ゴールデンボンバー
URL:     https://rateyourmusic.com/artist/ゴールデンボンバー
ID:      3178802068
Profile: {'profile': ' 2004, Tokyo, Japan', 'aliases': None, 'members': '鬼龍院翔 [Kiryuuin Shou] (vocals), 喜矢武豊 [Kyan Yutaka] (guitar), 歌広場 淳 [Utahiroba Jun] (bass), 樽美酒研二 [Darvish Kenji] (drums, 2009-present), 天空城団吉 [Tenkuujou Dankichi] (drums, 2007-09), 血祭鉄兵 [Chimatsuri Teppei] (drums, 2005-07)', 'sites': None, 'groups': None, 'variations': None, 'search': None, 'external': None, 'err': None}
Pages:   {'ppp': 1, 'tot': 1, 'pages': 1, 'err': None, 'more': False, 'redo': False}
Media:   {'counts': {'Releases': {'Album': 9, 'EP': 2, 'Single': 13, 'Appears On': 1, 'Compilation': 2, 'V/A Compilation': 2}}, 'err': None}
   Album
      The Golden J-Pops
      剃り残した夏 (Suri nokoshita natsu)
      Golden Best -Brassiere-
      ゴールデン・アワー ～上

In [22]:
from webUtils import removeTag

In [23]:
help(removeTag)

Help on function removeTag in module webUtils:

removeTag(line, tag, debug=False)
    ### BeautifulSoup functions



In [ ]:
url="https://genius.com/artists/Buddy"
data, code = downloadURL(url)
from ioUtils import saveFile, getFile
saveFile(idata=data, ifile="genius_buddy.p")

In [ ]:
bsdata = getHTML("genius_buddy.p")

In [ ]:
for meta in bsdata.findAll("meta"):
    content = meta.attrs['content']
    if content.startswith("{") and content.endswith("}"):
        jdata = json.loads(content)

In [ ]:
jdata.keys()

In [ ]:
albumData

In [ ]:

internationalList += ['Good Child Foundation', 'Neon Brotherhood', 'Rogue Souljahz']
internationalList += ['GMC', 'Krossfade', 'Dave Howley', 'Toko', 'Y-not', 'Afro Music']
internationalList += ['Husvenne', 'Maca Boy', 'Skyndeep', 'TwitterXmassingle', 'Julieanne Dineen', 'Julie-Anne Dineen', 'Anybodies']
internationalList += ["Miro"] # Bulgaria
internationalList += ["Lama", 'Vintaj', 'D-flow', 'U. Dumanska', 'Leonard']



In [ ]:
from dbArtistsGenius import dbArtistsGenius

In [3]:
#https://www.jiosaavn.com/song/terrestrial/HRovUgFpbl4


url= "https://genius.com/artists/Sanari"
url= "https://genius.com/artists/Kira-jp"
url= "https://genius.com/artists/Joipe"
url= "https://genius.com/artists/Aron-can"
url= "https://genius.com/artists/Herra-hnetusmjor"
url= "https://genius.com/artists/Omer-adam"
urls=[]
from time import sleep
#urls.append("https://genius.com/artists/Beastie-boys")
urls.append("https://genius.com/artists/Saske")
urls.append("https://genius.com/artists/Fy")
urls.append("https://genius.com/artists/Eden-ben-zaken")
urls.append("https://genius.com/artists/Itay-levi")
urls.append("https://genius.com/artists/Snik")
urls.append("https://genius.com/artists/Joey-christ")
urls=[] 
urls.append("https://genius.com/artists/Canozan")
urls.append("https://genius.com/artists/Kaya-giray")
urls.append("https://genius.com/yungouzo")
urls.append("https://genius.com/artists/Nahide-babashl")
urls.append("https://genius.com/artists/Brado")
urls.append("https://genius.com/artists/Narkoz")
urls.append("https://genius.com/artists/Gunay-aksoy")
urls=[]
urls.append("https://genius.com/artists/Vallis-alps")
urls.append("https://genius.com/artists/G-flip")
urls.append("https://genius.com/artists/Chasing-abbey")
urls.append("https://genius.com/artists/Hp-boyz")
urls.append("https://genius.com/artists/Youngn-lipz")
urls.append("https://genius.com/artists/No-money-enterprise")
urls.append("https://genius.com/artists/Louisa")
urls.append("https://genius.com/artists/Sigma")
urls.append("https://genius.com/artists/Cxloe")
urls.append("https://genius.com/artists/Mallrat")
urls.append("https://genius.com/artists/Tyron-hapi")
urls.append("https://genius.com/artists/Chillinit")
urls.append("https://genius.com/artists/Hooligan-hefs")
urls.append("https://genius.com/artists/Onefour")
urls=["https://genius.com/albums/Mudi/Amal", 
"https://genius.com/artists/I-waal", 
"https://genius.com/artists/Colin-firth", 
"https://genius.com/artists/5-after-midnight", 
"https://genius.com/artists/Sunset-bros-x-mark-mccabe", 
"https://genius.com/artists/Migos", 
"https://genius.com/artists/The-academic", 
"https://genius.com/artists/Josh-dylan", 
"https://genius.com/artists/Hugh-skinner", 
"https://genius.com/artists/Jeremy-irvine", 
"https://genius.com/artists/Offaiah", 
"https://genius.com/artists/The-crystals", 
"https://genius.com/artists/Jeremy-soule", 
"https://genius.com/artists/Tiagz", 
"https://genius.com/artists/Lil-xxel", 
"https://genius.com/artists/Irish-women-in-harmony", 
"https://genius.com/artists/Robert-grace", 
"https://genius.com/artists/Ryan-blyth", 
"https://genius.com/artists/Whtkd", 
"https://genius.com/artists/Pnl", 
"https://genius.com/artists/Slumberjack", 
"https://genius.com/artists/Passion", 
"https://genius.com/artists/Biggy", 
"https://genius.com/artists/Pon-cho", 
"https://genius.com/artists/Run-dmc", 
"https://genius.com/artists/Golden-features", 
"https://genius.com/artists/Dastic", 
"https://genius.com/artists/Kita-alexander", 
"https://genius.com/artists/Stace-cadet", 
"https://genius.com/artists/Carmouflage-rose", 
"https://genius.com/artists/Fergus-james", 
"https://genius.com/artists/Triple-one", 
"https://genius.com/artists/Day1", 
"https://genius.com/artists/Lyra", 
"https://genius.com/artists/Af1"]
urls=["https://genius.com/artists/Liga"]
urls=["https://genius.com/artists/Le-mo-dnk", 'https://genius.com/artists/Benee']
urls=["https://genius.com/artists/Passion-dnk", 'https://genius.com/artists/One-bit-and-noah-cyrus', 'https://genius.com/artists/Soule']
urls=["https://genius.com/artists/Rin", "https://genius.com/artists/Fler", "https://genius.com/artists/Gringo",
"https://genius.com/artists/Lil-lano", "https://genius.com/artists/Shirin-david", "https://genius.com/artists/Monet192",
"https://genius.com/artists/Kasimir1441", "https://genius.com/artists/Badmomzjay", "https://genius.com/artists/Jazn",
"https://genius.com/artists/Reezy", "https://genius.com/artists/King-khalil", "https://genius.com/artists/Celine",
"https://genius.com/artists/Bhz", "https://genius.com/artists/Xatar", "https://genius.com/artists/Finch-asozial",
"https://genius.com/artists/Hanybal", "https://genius.com/artists/Tj-beastboy", "https://genius.com/artists/Abdi",
"https://genius.com/artists/Fourty", "https://genius.com/artists/Sdp", "https://genius.com/artists/Ngee",
"https://genius.com/artists/Sinan-g", "https://genius.com/artists/Rais", "https://genius.com/artists/Sxtn",
"https://genius.com/artists/Ali471", "https://genius.com/artists/Reynmen", "https://genius.com/artists/Bozza",
"https://genius.com/artists/Metrickz", "https://genius.com/artists/Slavik", "https://genius.com/artists/Play69",
"https://genius.com/artists/Spongebozz", "https://genius.com/artists/Juri", "https://genius.com/artists/Saaftig",
"https://genius.com/artists/Mois", "https://genius.com/artists/Dhurata-dora", "https://genius.com/artists/Knossi","https://genius.com/artists/Gent"]
urls=["https://genius.com/artists/Casper", "https://genius.com/artists/Yonii", "https://genius.com/artists/Belah",
"https://genius.com/artists/Kayef", "https://genius.com/artists/Elif", "https://genius.com/artists/Deno419",
"https://genius.com/artists/Camila-cabello", "https://genius.com/artists/Garagen-larrys", "https://genius.com/artists/Beyazz",
"https://genius.com/artists/Firat", "https://genius.com/artists/Hitimpulse", "https://genius.com/artists/Sarhad",
"https://genius.com/artists/Ivana-santacruz", "https://genius.com/artists/Schubi-akpella", "https://genius.com/artists/Soufian",
"https://genius.com/artists/Kilomatik", "https://genius.com/artists/Tarek-kiz", "https://genius.com/artists/Tayna",
"https://genius.com/artists/Die-schwarzwalder-kirschtorten", "https://genius.com/artists/Gary-washington", "https://genius.com/artists/Whethan",
"https://genius.com/artists/Anstandslos-and-durchgeknallt", "https://genius.com/artists/Jay-samuelz", "https://genius.com/artists/Sugar-mmfk",
"https://genius.com/artists/Delil", "https://genius.com/artists/Apored", "https://genius.com/artists/Payy",
"https://genius.com/artists/Joshi-mizu", "https://genius.com/artists/Mike-singer", "https://genius.com/artists/Naaz",
"https://genius.com/artists/Luqe", "https://genius.com/artists/Laruzo", "https://genius.com/artists/Sarah-lombardi",
"https://genius.com/artists/Tom-gregory", "https://genius.com/artists/Fake-pictures", "https://genius.com/artists/Ghetto-phenomene"]
urls=["https://genius.com/artists/Dante-yn", "https://genius.com/artists/Reda-rwena", "https://genius.com/artists/Remoe",
     "https://genius.com/artists/Lucky-luke", "https://genius.com/artists/Paula-dalla-corte", "https://genius.com/artists/Inoffiziellgoldenboy",
     "https://genius.com/artists/Melo68", "https://genius.com/artists/Casar", "https://genius.com/artists/Aylo",
     "https://genius.com/artists/Lune", "https://genius.com/artists/Phil-the-beat", "https://genius.com/artists/Amanda-delara",
     "https://genius.com/artists/Monk", "https://genius.com/artists/Thedodo", "https://genius.com/artists/Harris-and-ford",
     "https://genius.com/artists/Kidda", "https://genius.com/artists/Georg-stengel", "https://genius.com/artists/Qzeng",
     "https://genius.com/artists/Hugel", "https://genius.com/artists/Querbeat", "https://genius.com/artists/Kynda-gray",
     "https://genius.com/artists/Estikay", "https://genius.com/artists/Hemso", "https://genius.com/artists/Patron",
     "https://genius.com/artists/Raf-camora", "https://genius.com/artists/Undacava", "https://genius.com/artists/Jiggo"]
urls=["https://genius.com/artists/Yung-kafa-and-kucuk-efendi"]
urls=["https://genius.com/artists/Powerkryner", "https://genius.com/artists/Stupid-goldfish", "https://genius.com/artists/Yassazin",
 "https://genius.com/artists/Avec", "https://genius.com/artists/Anastasija", "https://genius.com/artists/Kyd-the-band",
 "https://genius.com/artists/Stefan-rauch", "https://genius.com/artists/Ahmad-amin", "https://genius.com/artists/Masn",
 "https://genius.com/artists/Greeen", "https://genius.com/artists/Mc-stojan", "https://genius.com/artists/Alle-achtung",
 "https://genius.com/artists/Devito", "https://genius.com/artists/Relja", "https://genius.com/artists/Chris-steger",
 "https://genius.com/artists/Nazar", "https://genius.com/artists/Sultan"]
urls=['https://genius.com/artists/Nightcall', 'https://genius.com/artists/Darius-and-finlay']
for url in urls:
    print(url)
    dp = dbArtistsGenius()
    dp.downloadArtistFromURL(url)
    sleep(2)
#artistID = dp.dutils.getArtistID(url)
#savename = dp.dutils.getArtistSavename(artistID)
#print(url,savename)

https://genius.com/artists/Nightcall
=========================   genius   =========================
Saving /Volumes/Piggy/Discog/artists-genius/87/41141987.p (force=False)
  --> This file is 51.9kB.
Done. Sleeping for 2 seconds
https://genius.com/artists/Darius-and-finlay
=========================   genius   =========================
Saving /Volumes/Piggy/Discog/artists-genius/79/54344879.p (force=False)
  --> This file is 52.2kB.
Done. Sleeping for 2 seconds


In [ ]:
url="https://genius.com/search?q=Dave%20Matthews%20Band"
data, code = downloadURL(url)
saveFile(idata=data, ifile="genius_search.p")

In [ ]:
bsdata = getHTML("genius_search.p")

In [ ]:
https://kworb.net/spotify/track/5NJKIFLOLa6Oc9auYeJmtx.html

In [ ]:
%autoreload
ag = artistGenius()
retval = ag.getData("genius_buddy.p")
retval.show()

In [ ]:
ai = artistIHeart()
retval = ai.getData("iheart_albums.p")
retval.show()

In [ ]:
from webUtils import getHTML
bsdata = getHTML("iheart_albums.p")

In [ ]:
x = 'https://www.iheart.com/artist/helgi-bjornsson-32568452/albums/'
x[:-8]

In [ ]:
bsdata

In [ ]:
script = bsdata.find("script", {"data-name": "initial-state"})
import json
jdata = json.loads(script.contents[0])

In [ ]:
import datetime
for artistID, artistData in jdata['artists']['artists'].items():
    artistName = artistData["artistName"]
    artistBio  = artistData["artistBio"]
    albums     = artistData["albums"]
    tracks     = artistData["tracks"]
    print(artistName)
    print(artistBio)
    
    mediaType = "Albums"
    for albumData in albums:
        albumID   = albumData['albumId']
        albumName = albumData['title']
        albumEpoch = albumData['releaseDate']
        #dt = datetime.datetime.fromtimestamp(albumEpoch)
        print(albumName,albumID,albumEpoch)
        
    mediaType = "Singles"
    for trackData in tracks:
        trackID   = trackData['trackId']
        trackName = trackData['title']
        print(trackName,trackID)

In [ ]:
for artistID,artistData in jdata['albums'].items():
    if artistID == "albums":
        continue
    print(artistID)
    albumsData = artistData["albums"]
    for albumData in albumsData:
        albumID     = albumData["albumId"]
        artistName  = albumData["artistName"]
        albumName   = albumData["title"]
        albumTracks = albumData["tracks"]
        
        tracks = []
        for track in albumTracks:
            trackID    = track['id']
            trackTitle = track['title']
            trackPos   = track['albumInfo'].get('trackNumber')
            try:
                int(trackPos)
            except:
                trackPos = -1
            
            tracks.append({"ID": trackID, "Title": trackTitle, "Pos": trackPos})

        print(albumName,'\t',len(tracks))



In [ ]:
from webUtils import getHTML
bsdata = getHTML("iheart_songs.p")

In [ ]:
script = bsdata.find("script", {"data-name": "initial-state"})
import json
jdata = json.loads(script.contents[0])

In [ ]:
jdata

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
dbAP = dbArtistsDeezer()
dbAP.searchForArtist("Gilli", force=False,maxArtists=1,debug=True)

In [ ]:
from musicDBMap import musicDBMap
mdbmap = musicDBMap("Music", init=False, copy=False)

In [ ]:
from time import sleep
for i,(primaryKey,artistName) in enumerate(mdbmap.getArtists().items()):
    if i < 19:
        continue
    dbAP = dbArtistsDeezer()
    print("i = {0}".format(i))
    dbAP.searchForArtist(artistName,force=False,maxArtists=1,debug=True)
    sleep(3.5)

In [ ]:
mdbmap = musicDBMap("Music")

In [ ]:
df = mdbmap.getDF()

In [ ]:
toget={"Deezer": [], "LastFM": []}
for primaryKey,artistName in mdbmap.getArtists().items():
    artistData = mdbmap.getArtistDataByName(artistName).getDict()
    if artistData.get('Deezer') is None:
        toget["Deezer"].append(artistName)
    if artistData.get('LastFM') is None:
        toget["LastFM"].append(artistName)

In [ ]:
toget["Deezer"]

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
from dbArtistsLastFM import dbArtistsLastFM

from time import sleep
for i,(primaryKey,artistName) in enumerate(mdbmap.getArtists().items()):
    if i < 551:
        continue
    print("="*50,artistName,"="*50)
    print("i = {0}".format(i))
    dbAP = dbArtistsLastFM()
    dbAP.searchForArtist(artistName,force=False,maxArtists=1,debug=True)
    sleep(1.5)
    
    dbAP = dbArtistsDeezer()
    dbAP.searchForArtist(artistName,force=False,maxArtists=1,debug=True)
    sleep(1.5)

In [ ]:
data, code = downloadURL("https://www.deezer.com/search/Benny%20Jamz")
saveFile(idata=data, ifile="deezer_search.p")

In [ ]:
from ioUtils import saveFile, getFile
saveFile(idata=data, ifile="deezer_test.p")

In [ ]:
from webUtils import getHTML
bsdata = getHTML("deezer_test.p")

In [ ]:
sdata = getHTML("deezer_search.p")

In [ ]:
import json
for script in sdata.findAll("script"):
    if len(script.contents) == 0:
        continue
        
    if script.contents[0].startswith("window.__DZR_APP_STATE__ = "):        
        try:        
            jdata = json.loads(script.contents[0].replace("window.__DZR_APP_STATE__ = ", ""))
            print("HI")
        except:
            continue

In [ ]:
for result in jdata["TOP_RESULT"]:
    artistID   = result["ART_ID"]
    artistName = result["ART_NAME"]
    
    url = "https://www.deezer.com/us/artist/{0}/top_track".format(artistID)
    savename = "{0}_{1}.p".format(artistID, artistName)
    data, code = downloadURL(url)
    saveFile(idata=data, ifile=savename, debug=True)

In [ ]:
%load_ext autoreload
%autoreload
from artistDeezer import artistDeezer
ad = artistDeezer()
retval = ad.getData("/Volumes/Piggy/Discog/artists-deezer/80/6980.p")
retval.show()

In [ ]:
bsdata = getHTML("5395257_Benny Jamz.p")
bsdata

<h1 class="heading-1" id="naboo_artist_name">
<a class="link-gray-light" href="/us/artist/5395257">
<span itemprop="name">Benny Jamz</span>
</a>
</h1>

In [ ]:
for result in jdata["ARTIST"]['data']:    
    artistID   = result["ART_ID"]
    artistName = result["ART_NAME"]

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dbAP.parse(modVal, previousDays=1, force=False)
    
for modVal in range(100):
    break
    dbAP = dbArtistsExtra(dbArtistsDiscogs())
    dbAP.parse(modVal, previousDays=10, force=True)
    
for modVal in range(100):
    break
    dbAP = dbArtistsUnofficial(dbArtistsDiscogs())
    dbAP.parse(modVal, previousDays=10, force=True)
    
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsAssertUnofficial(dbArtistsDiscogs())
    dbAP.parseModVal(modVal)
    dbAP.downloadMissingArtistUnofficials()

# AllMusic

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertCredit

for modVal in range(65,75):
    dbAP = dbArtistsCredit(dbArtistsAllMusic())
    dbAP.parse(modVal, previousDays=100)

In [ ]:
vals="""=========================   discogs   =========================
  ===> Found 7566 previous data for ModVal=0
Found 336 artists to get
Error downloading https://www.discogs.com/artist/1439900-Adrian-Lux?type=Unofficial
Error downloading https://www.discogs.com/artist/2372000-Blasfemia-3?type=Unofficial
Error downloading https://www.discogs.com/artist/4353300-Keith-Jarrett-Gary-Peacock-Jack-DeJohnette?type=Unofficial
Error downloading https://www.discogs.com/artist/493200-Glennis-Grace?type=Unofficial
Error downloading https://www.discogs.com/artist/582500-Zara-2?type=Unofficial
Error downloading https://www.discogs.com/artist/718900-G-Punkt-2?type=Unofficial
=========================   discogs   =========================
  ===> Found 7844 previous data for ModVal=1
Found 347 artists to get
Error downloading https://www.discogs.com/artist/137201-SLM?type=Unofficial
Error downloading https://www.discogs.com/artist/212401-Powerman?type=Unofficial
Error downloading https://www.discogs.com/artist/2428401-Horse-Head-2?type=Unofficial
Error downloading https://www.discogs.com/artist/29101-DONS?type=Unofficial
Error downloading https://www.discogs.com/artist/3853201-Svolder?type=Unofficial
Error downloading https://www.discogs.com/artist/4303701-Pera-Joe-Blues-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/5277801-Orquesta-Imperial-Vienesa?type=Unofficial
Error downloading https://www.discogs.com/artist/68501-Colonel-Mite?type=Unofficial
Error downloading https://www.discogs.com/artist/75001-GTR?type=Unofficial
=========================   discogs   =========================
  ===> Found 7601 previous data for ModVal=2
Found 322 artists to get
Error downloading https://www.discogs.com/artist/2396602-The-Jimmy-Johnson-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/2963602-Johnny-Oliver?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/2/unofficial/309502.p (force=False)
  --> This file is 17.2kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7870 previous data for ModVal=3
Found 362 artists to get
Error downloading https://www.discogs.com/artist/1154403-Tiny-Hat-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/271303-DJ-Harry-2?type=Unofficial
Error downloading https://www.discogs.com/artist/2750003-The-New-Eyes?type=Unofficial
Error downloading https://www.discogs.com/artist/303-Future-Forces-Inc?type=Unofficial
Error downloading https://www.discogs.com/artist/356103-The-Executioners-2?type=Unofficial
Error downloading https://www.discogs.com/artist/4760503-Roger-Waters-Eric-Clapton?type=Unofficial
Error downloading https://www.discogs.com/artist/5652703-We-Love-Beats?type=Unofficial
Error downloading https://www.discogs.com/artist/7303-Aftershock?type=Unofficial
Error downloading https://www.discogs.com/artist/830403-RPM-3?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/3/unofficial/2782403.p (force=False)
  --> This file is 17.4kB.
Done. Sleeping for 2 seconds
Saving /Volumes/Piggy/Discog/artists-discogs/3/unofficial/4985803.p (force=False)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7695 previous data for ModVal=4
Found 314 artists to get
Error downloading https://www.discogs.com/artist/1390704-Marina-Del-Ray?type=Unofficial
Error downloading https://www.discogs.com/artist/14904-Rhythmatic?type=Unofficial
Error downloading https://www.discogs.com/artist/1684304-Bobby-Krlic?type=Unofficial
Error downloading https://www.discogs.com/artist/210204-The-Rush?type=Unofficial
Error downloading https://www.discogs.com/artist/2570204-James-Last-His-Orchestra-And-Singers?type=Unofficial
Error downloading https://www.discogs.com/artist/32904-Paul-Newman?type=Unofficial
Error downloading https://www.discogs.com/artist/4005604-Yuko-Ishibashi?type=Unofficial
Error downloading https://www.discogs.com/artist/4681404-Lawrence-Brownlee?type=Unofficial
Error downloading https://www.discogs.com/artist/730304-Alfred-Scholz?type=Unofficial
Error downloading https://www.discogs.com/artist/872704-Matsuo-Ono?type=Unofficial
Error downloading https://www.discogs.com/artist/922704-Munir-Bashir?type=Unofficial
Error downloading https://www.discogs.com/artist/940904-Cool-Kids?type=Unofficial
Error downloading https://www.discogs.com/artist/3694504-Cloaca-Maxima?type=Unofficial
=========================   discogs   =========================
  ===> Found 7527 previous data for ModVal=5
Found 344 artists to get
Error downloading https://www.discogs.com/artist/186105-Taraf-de-Ha%C3%AFdouks?type=Unofficial
Error downloading https://www.discogs.com/artist/4005605-Rashufel?type=Unofficial
Error downloading https://www.discogs.com/artist/407805-Carla-White?type=Unofficial
Error downloading https://www.discogs.com/artist/490405-WW-III?type=Unofficial
=========================   discogs   =========================
  ===> Found 7558 previous data for ModVal=6
Found 329 artists to get
Error downloading https://www.discogs.com/artist/11006-TJR?type=Unofficial
Error downloading https://www.discogs.com/artist/1122606-Paul-Lavalle?type=Unofficial
Error downloading https://www.discogs.com/artist/173406-DJ-EmBee?type=Unofficial
Error downloading https://www.discogs.com/artist/255106-Deja-Gruv?type=Unofficial
Error downloading https://www.discogs.com/artist/312006-Andy-Fisher?type=Unofficial
Error downloading https://www.discogs.com/artist/3832906-Golden-Sounds-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/538306-The-Saliva-Commandos?type=Unofficial
Error downloading https://www.discogs.com/artist/6505806-National-Skin?type=Unofficial
Error downloading https://www.discogs.com/artist/745606-The-Art-Van-Damme-Quintet?type=Unofficial
Error downloading https://www.discogs.com/artist/964306-Angelyne?type=Unofficial
=========================   discogs   =========================
  ===> Found 7661 previous data for ModVal=7
Found 311 artists to get
Error downloading https://www.discogs.com/artist/1532407-Steve-Hitchcock-2?type=Unofficial
Error downloading https://www.discogs.com/artist/1570807-The-Hi-Liters?type=Unofficial
Error downloading https://www.discogs.com/artist/189507-Virtuoso-2?type=Unofficial
Error downloading https://www.discogs.com/artist/2085607-Raphy-Leavitt-Y-Su-Orquesta-La-Selecta?type=Unofficial
Error downloading https://www.discogs.com/artist/2866207-2nd-Family?type=Unofficial
Error downloading https://www.discogs.com/artist/311107-Chris-Parker-2?type=Unofficial
Error downloading https://www.discogs.com/artist/5422807-Die-Laughing-5?type=Unofficial
Error downloading https://www.discogs.com/artist/611007-El-Pino-and-the-Volunteers?type=Unofficial
Error downloading https://www.discogs.com/artist/623607-David-Maxwell?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/7/unofficial/2063207.p (force=False)
  --> This file is 16.5kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7711 previous data for ModVal=8
Found 345 artists to get
Error downloading https://www.discogs.com/artist/13808-Boyd-Jarvis?type=Unofficial
Error downloading https://www.discogs.com/artist/15808-Peter-Howell?type=Unofficial
Error downloading https://www.discogs.com/artist/2666808-Seaside-Lovers?type=Unofficial
Error downloading https://www.discogs.com/artist/2884808-Luder?type=Unofficial
Error downloading https://www.discogs.com/artist/337908-Sugar-Jones?type=Unofficial
Error downloading https://www.discogs.com/artist/84408-Robbie?type=Unofficial
=========================   discogs   =========================
  ===> Found 7616 previous data for ModVal=9
Found 365 artists to get
Error downloading https://www.discogs.com/artist/1534909-Catherine-Ribeiro-Alpes?type=Unofficial
Error downloading https://www.discogs.com/artist/17609-Liaisons-D?type=Unofficial
Error downloading https://www.discogs.com/artist/303209-Machito?type=Unofficial
Error downloading https://www.discogs.com/artist/95409-James-Yorkston?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/9/unofficial/1350809.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7641 previous data for ModVal=10
Found 349 artists to get
Error downloading https://www.discogs.com/artist/1112110-Wally-Gonzales?type=Unofficial
Error downloading https://www.discogs.com/artist/114710-Dustin-Zahn?type=Unofficial
Error downloading https://www.discogs.com/artist/1574510-Pogo-11?type=Unofficial
Error downloading https://www.discogs.com/artist/1779910-Ekoplekz?type=Unofficial
Error downloading https://www.discogs.com/artist/1904310-Yukio-Hashi?type=Unofficial
Error downloading https://www.discogs.com/artist/2048710-Ranfold-Williams?type=Unofficial
Error downloading https://www.discogs.com/artist/21210-Ali-B?type=Unofficial
Error downloading https://www.discogs.com/artist/2230410-Conjunto-Black-Power?type=Unofficial
Error downloading https://www.discogs.com/artist/225210-Minor-Detail?type=Unofficial
Error downloading https://www.discogs.com/artist/262610-TISM?type=Unofficial
Error downloading https://www.discogs.com/artist/2627610-Unknown-40?type=Unofficial
Error downloading https://www.discogs.com/artist/310-Decoder?type=Unofficial
Error downloading https://www.discogs.com/artist/898410-Stephen-Keyes?type=Unofficial
=========================   discogs   =========================
  ===> Found 7671 previous data for ModVal=11
Found 341 artists to get
Error downloading https://www.discogs.com/artist/261511-Sponge-3?type=Unofficial
Error downloading https://www.discogs.com/artist/278011-The-Skeptix?type=Unofficial
Error downloading https://www.discogs.com/artist/3371411-Set-It-Off-2?type=Unofficial
Error downloading https://www.discogs.com/artist/4704511-Brett-Ellis-2?type=Unofficial
Error downloading https://www.discogs.com/artist/5270211-Tomasz-Sta%C3%85%C2%84ko-W%C3%85%C2%82adys%C3%85%C2%82aw-Sendecki?type=Unofficial
Error downloading https://www.discogs.com/artist/5811811-The-Bachelors-11?type=Unofficial
Error downloading https://www.discogs.com/artist/66611-Mash?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/11/unofficial/262411.p (force=False)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7712 previous data for ModVal=12
Found 331 artists to get
Error downloading https://www.discogs.com/artist/1136212-Georgia-Carr?type=Unofficial
Error downloading https://www.discogs.com/artist/1604012-First-Rebirth?type=Unofficial
Error downloading https://www.discogs.com/artist/166812-Michael-Mann?type=Unofficial
Error downloading https://www.discogs.com/artist/1909412-Ranma-Operatic-Troupe?type=Unofficial
Error downloading https://www.discogs.com/artist/2689812-Willburn-Brothers?type=Unofficial
Error downloading https://www.discogs.com/artist/2727912-Les-T%C3%AAtes-Blanches?type=Unofficial
Error downloading https://www.discogs.com/artist/4955912-EAL-Axner?type=Unofficial
Error downloading https://www.discogs.com/artist/5475512-SDP-5?type=Unofficial
Error downloading https://www.discogs.com/artist/84912-Mutter?type=Unofficial
=========================   discogs   =========================
  ===> Found 7776 previous data for ModVal=13
Found 360 artists to get
Error downloading https://www.discogs.com/artist/21913-T-La-Rock?type=Unofficial
Error downloading https://www.discogs.com/artist/2446213-The-Amity-Affliction?type=Unofficial
Error downloading https://www.discogs.com/artist/2639413-ChefSpecial?type=Unofficial
Error downloading https://www.discogs.com/artist/283913-The-Kilimanjaro-Darkjazz-Ensemble?type=Unofficial
Error downloading https://www.discogs.com/artist/3231613-DJ-%D0%90%D0%BD%D0%B6%D0%B5%D0%BB%D0%B0-%D0%A8%D1%83%D0%BB%D1%8C%D0%B6%D0%B5%D0%BD%D0%BA%D0%BE?type=Unofficial
Error downloading https://www.discogs.com/artist/36113-You?type=Unofficial
Error downloading https://www.discogs.com/artist/39713-Edan?type=Unofficial
Error downloading https://www.discogs.com/artist/5041113-Silk-N-Steel?type=Unofficial
Error downloading https://www.discogs.com/artist/70213-Mr-Gasmask?type=Unofficial
Error downloading https://www.discogs.com/artist/862913-Francisco-T%C3%A1rrega?type=Unofficial
=========================   discogs   =========================
  ===> Found 7527 previous data for ModVal=14
Found 300 artists to get
Error downloading https://www.discogs.com/artist/1017114-David-Tattersall?type=Unofficial
Error downloading https://www.discogs.com/artist/1147814-Cities-Service-Band-Of-America?type=Unofficial
Error downloading https://www.discogs.com/artist/115614-Yuka?type=Unofficial
Error downloading https://www.discogs.com/artist/13914-DJ-Trajic?type=Unofficial
Error downloading https://www.discogs.com/artist/346214-Mr-Review?type=Unofficial
Error downloading https://www.discogs.com/artist/6869914-Fastfinger-Smith?type=Unofficial
=========================   discogs   =========================
  ===> Found 7641 previous data for ModVal=15
Found 312 artists to get
Error downloading https://www.discogs.com/artist/166415-HOP?type=Unofficial
Error downloading https://www.discogs.com/artist/380215-DBN?type=Unofficial
Error downloading https://www.discogs.com/artist/3872015-Eddie-Lebron-And-His-Orch?type=Unofficial
Error downloading https://www.discogs.com/artist/39915-Flesh-Field?type=Unofficial
Error downloading https://www.discogs.com/artist/479215-Xalan?type=Unofficial
=========================   discogs   =========================
  ===> Found 7576 previous data for ModVal=16
Found 339 artists to get
Error downloading https://www.discogs.com/artist/176716-Big-B?type=Unofficial
Error downloading https://www.discogs.com/artist/190016-Manuel-Mota?type=Unofficial
Error downloading https://www.discogs.com/artist/19216-Indo?type=Unofficial
Error downloading https://www.discogs.com/artist/216-Club-69?type=Unofficial
Error downloading https://www.discogs.com/artist/355316-Say-When%21?type=Unofficial
Error downloading https://www.discogs.com/artist/4200216-Paul-Page-5?type=Unofficial
Error downloading https://www.discogs.com/artist/46716-Dom-Salvador-e-Aboli%C3%A7ao?type=Unofficial
Error downloading https://www.discogs.com/artist/4954516-Del-Larks?type=Unofficial
Error downloading https://www.discogs.com/artist/6163416-Manekeni?type=Unofficial
Error downloading https://www.discogs.com/artist/655616-Jerome-Moross?type=Unofficial
Error downloading https://www.discogs.com/artist/667216-Peter-Tork?type=Unofficial
Error downloading https://www.discogs.com/artist/695016-Zoogz-Rift?type=Unofficial
=========================   discogs   =========================
  ===> Found 7658 previous data for ModVal=17
Found 331 artists to get
Error downloading https://www.discogs.com/artist/1004517-Truckfighters?type=Unofficial
Error downloading https://www.discogs.com/artist/136017-Ann-Robinson?type=Unofficial
Error downloading https://www.discogs.com/artist/1700517-Madame-Blavatsky?type=Unofficial
Error downloading https://www.discogs.com/artist/2608717-Ralph-Lundsten-And-The-Andromeda-All-Stars?type=Unofficial
Error downloading https://www.discogs.com/artist/2690617-Northlane?type=Unofficial
Error downloading https://www.discogs.com/artist/28817-Brandy?type=Unofficial
Error downloading https://www.discogs.com/artist/330017-Mac-Dre?type=Unofficial
Error downloading https://www.discogs.com/artist/3495917-Doris-Browne?type=Unofficial
Error downloading https://www.discogs.com/artist/374717-Ville-Valo?type=Unofficial
Error downloading https://www.discogs.com/artist/4180617-General-Store?type=Unofficial
Error downloading https://www.discogs.com/artist/430217-Greg-Van-Bueren?type=Unofficial
=========================   discogs   =========================
  ===> Found 7687 previous data for ModVal=18
Found 347 artists to get
Error downloading https://www.discogs.com/artist/119518-Yuksek?type=Unofficial
Error downloading https://www.discogs.com/artist/1320718-Werner-Stiefel?type=Unofficial
Error downloading https://www.discogs.com/artist/1471218-EAST-6?type=Unofficial
Error downloading https://www.discogs.com/artist/165718-Stanley-Foort?type=Unofficial
Error downloading https://www.discogs.com/artist/231818-Santos-3?type=Unofficial
Error downloading https://www.discogs.com/artist/24018-Cyba-Space?type=Unofficial
Error downloading https://www.discogs.com/artist/2519018-FM-Freeks?type=Unofficial
Error downloading https://www.discogs.com/artist/362918-Vic-Rawlings?type=Unofficial
Error downloading https://www.discogs.com/artist/475918-Louis-Neefs?type=Unofficial
Error downloading https://www.discogs.com/artist/4889118-Alt%C3%BA-Pag%C3%A1n%C3%A1ch?type=Unofficial
Error downloading https://www.discogs.com/artist/60718-DJ-Vas?type=Unofficial
=========================   discogs   =========================
  ===> Found 7624 previous data for ModVal=19
Found 356 artists to get
Error downloading https://www.discogs.com/artist/2198919-Antiquus-Scriptum?type=Unofficial
Error downloading https://www.discogs.com/artist/230519-Nicolay?type=Unofficial
Error downloading https://www.discogs.com/artist/2567719-Eden-44?type=Unofficial
Error downloading https://www.discogs.com/artist/3110519-Lloyd-Sims?type=Unofficial
Error downloading https://www.discogs.com/artist/3881519-The-Chorus-And-Orchestra-Of-The-Metropolitan-Opera-Association?type=Unofficial
Error downloading https://www.discogs.com/artist/4934319-Goran-Bregovi%C3%84%C2%87s-Karmen?type=Unofficial
Error downloading https://www.discogs.com/artist/5890419-Rita-Sakura?type=Unofficial
Error downloading https://www.discogs.com/artist/60419-Ruff-With-The-Smooth?type=Unofficial
Error downloading https://www.discogs.com/artist/68219-The-Masterdon-Committee?type=Unofficial
=========================   discogs   =========================
  ===> Found 7678 previous data for ModVal=20
Found 370 artists to get
Error downloading https://www.discogs.com/artist/1210920-Royce-Campbell?type=Unofficial
Error downloading https://www.discogs.com/artist/1358820-Nick-Curran?type=Unofficial
Error downloading https://www.discogs.com/artist/2120-Capone?type=Unofficial
Error downloading https://www.discogs.com/artist/2151720-John-Lundvik?type=Unofficial
Error downloading https://www.discogs.com/artist/4220-X?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/20/unofficial/4723220.p (force=False)
  --> This file is 21.3kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/49020-Moksha?type=Unofficial
Error downloading https://www.discogs.com/artist/504620-Blood-Of-Christ?type=Unofficial
Error downloading https://www.discogs.com/artist/637620-Henry-Mancini-And-His-Orchestra-And-Chorus?type=Unofficial
Error downloading https://www.discogs.com/artist/660620-Chuck-Ragan?type=Unofficial
Error downloading https://www.discogs.com/artist/750620-Suze?type=Unofficial
=========================   discogs   =========================
  ===> Found 7627 previous data for ModVal=21
Found 320 artists to get
Error downloading https://www.discogs.com/artist/1215121-The-Rising-Storm?type=Unofficial
Error downloading https://www.discogs.com/artist/1222921-Queen-Paul-Rodgers?type=Unofficial
Error downloading https://www.discogs.com/artist/2207421-Sorcery-3?type=Unofficial
Error downloading https://www.discogs.com/artist/2220821-Most-Dope?type=Unofficial
Error downloading https://www.discogs.com/artist/2709921-IOEarth?type=Unofficial
Error downloading https://www.discogs.com/artist/284821-Leah-Buckareff?type=Unofficial
Error downloading https://www.discogs.com/artist/5076121-The-Soulettes-5?type=Unofficial
Error downloading https://www.discogs.com/artist/728821-Postage?type=Unofficial
Error downloading https://www.discogs.com/artist/74321-Trickster-CMVs?type=Unofficial
Error downloading https://www.discogs.com/artist/801621-Sonic-Reducer?type=Unofficial
=========================   discogs   =========================
  ===> Found 7650 previous data for ModVal=22
Found 360 artists to get
Error downloading https://www.discogs.com/artist/2070422-Ravi-Shankar-Ali-Akbar-Khan?type=Unofficial
Error downloading https://www.discogs.com/artist/2294222-Dopethrone?type=Unofficial
Error downloading https://www.discogs.com/artist/269222-Ahmed-Abdullah?type=Unofficial
Error downloading https://www.discogs.com/artist/305622-Pluton-Svea?type=Unofficial
Error downloading https://www.discogs.com/artist/348622-Mike-Czech?type=Unofficial
Error downloading https://www.discogs.com/artist/3886122-Tony-Key?type=Unofficial
Error downloading https://www.discogs.com/artist/4753422-The-Chateaus-4?type=Unofficial
Error downloading https://www.discogs.com/artist/518522-Crowpath?type=Unofficial
Error downloading https://www.discogs.com/artist/67822-MR?type=Unofficial
Error downloading https://www.discogs.com/artist/755522-Peggy-Seeger?type=Unofficial
=========================   discogs   =========================
  ===> Found 7656 previous data for ModVal=23
Found 335 artists to get
Error downloading https://www.discogs.com/artist/230723-Taxman-2?type=Unofficial
Error downloading https://www.discogs.com/artist/23223-JD-Braithwaite?type=Unofficial
Error downloading https://www.discogs.com/artist/2639123-JJ-Jellybean?type=Unofficial
Error downloading https://www.discogs.com/artist/277223-Anarchist-Academy?type=Unofficial
Error downloading https://www.discogs.com/artist/4563723-Dealey-Plaza?type=Unofficial
Error downloading https://www.discogs.com/artist/586723-Jeremy-Fish?type=Unofficial
=========================   discogs   =========================
  ===> Found 7458 previous data for ModVal=24
Found 302 artists to get
Error downloading https://www.discogs.com/artist/1217124-Salvation-Army-New-York-Staff-Band-And-Male-Chorus?type=Unofficial
Error downloading https://www.discogs.com/artist/2480624-Violet-Poison?type=Unofficial
Error downloading https://www.discogs.com/artist/306524-Willie-Alexander?type=Unofficial
Error downloading https://www.discogs.com/artist/328224-Rajoitus?type=Unofficial
Error downloading https://www.discogs.com/artist/4124-DJ-Friction?type=Unofficial
Error downloading https://www.discogs.com/artist/44024-Hurricane?type=Unofficial
Error downloading https://www.discogs.com/artist/4628024-The-Void-Pacific-Choir?type=Unofficial
Error downloading https://www.discogs.com/artist/6459024-Saved-My-Life?type=Unofficial
Error downloading https://www.discogs.com/artist/6690124-Guerilla-Tek?type=Unofficial
Error downloading https://www.discogs.com/artist/789424-Carmine-Appice-Project?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/24/unofficial/21724.p (force=False)
  --> This file is 17.1kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7632 previous data for ModVal=25
Found 314 artists to get
Error downloading https://www.discogs.com/artist/10825-Sugizo?type=Unofficial
Error downloading https://www.discogs.com/artist/181225-Nouri?type=Unofficial
Error downloading https://www.discogs.com/artist/252025-Pat-Patrick?type=Unofficial
Error downloading https://www.discogs.com/artist/5258925-Small-Black-Lambs?type=Unofficial
Error downloading https://www.discogs.com/artist/666425-Ryan-Unks?type=Unofficial
Error downloading https://www.discogs.com/artist/92825-The-Warlocks?type=Unofficial
=========================   discogs   =========================
  ===> Found 7723 previous data for ModVal=26
Found 317 artists to get
Error downloading https://www.discogs.com/artist/1277826-Vega-15?type=Unofficial
Error downloading https://www.discogs.com/artist/1574126-Doctor-Stereo?type=Unofficial
Error downloading https://www.discogs.com/artist/252526-The-Daily-Flash?type=Unofficial
Error downloading https://www.discogs.com/artist/288726-Moss-6?type=Unofficial
Error downloading https://www.discogs.com/artist/374226-Plies?type=Unofficial
Error downloading https://www.discogs.com/artist/422626-Sad-2?type=Unofficial
Error downloading https://www.discogs.com/artist/477926-AOK-3?type=Unofficial
Error downloading https://www.discogs.com/artist/48426-20th-Century-Steel-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/55326-Andrew-Chalk?type=Unofficial
Error downloading https://www.discogs.com/artist/629226-The-Longcut?type=Unofficial
Error downloading https://www.discogs.com/artist/67226-Stacey-Pullen?type=Unofficial
Error downloading https://www.discogs.com/artist/88426-Danny-Jay?type=Unofficial
=========================   discogs   =========================
  ===> Found 7546 previous data for ModVal=27
Found 317 artists to get
Saving /Volumes/Piggy/Discog/artists-discogs/27/unofficial/1583727.p (force=False)
  --> This file is 19.6kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/1727-Special-K?type=Unofficial
Error downloading https://www.discogs.com/artist/2056527-The-Hepsters?type=Unofficial
Error downloading https://www.discogs.com/artist/232827-Miss-Jones-2?type=Unofficial
Error downloading https://www.discogs.com/artist/281527-Luciano-Michelini?type=Unofficial
Error downloading https://www.discogs.com/artist/386327-Jim-Pembroke?type=Unofficial
Error downloading https://www.discogs.com/artist/5093227-Zac-Master?type=Unofficial
Error downloading https://www.discogs.com/artist/70427-Speech-Defect?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/27/unofficial/2949127.p (force=False)
  --> This file is 16.4kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7677 previous data for ModVal=28
Found 353 artists to get
Saving /Volumes/Piggy/Discog/artists-discogs/28/unofficial/1249728.p (force=False)
  --> This file is 27.7kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/127628-Mark-Robinson?type=Unofficial
Error downloading https://www.discogs.com/artist/1528-Abe-Duque?type=Unofficial
Error downloading https://www.discogs.com/artist/172128-The-Firm?type=Unofficial
Error downloading https://www.discogs.com/artist/295028-Wang-Fu-Ling?type=Unofficial
Error downloading https://www.discogs.com/artist/397328-Millie-Small?type=Unofficial
Error downloading https://www.discogs.com/artist/659328-The-King-Khan-BBQ-Show?type=Unofficial
Error downloading https://www.discogs.com/artist/82828-Android-Lust?type=Unofficial
Error downloading https://www.discogs.com/artist/1489428-Tony-Dangerfield-The-Thrills?type=Unofficial
=========================   discogs   =========================
  ===> Found 7581 previous data for ModVal=29
Found 310 artists to get
Error downloading https://www.discogs.com/artist/20229-Technique?type=Unofficial
Error downloading https://www.discogs.com/artist/218429-Inoyama-Land?type=Unofficial
Error downloading https://www.discogs.com/artist/2227229-Ray-Conniff-His-Singers-His-Orchestra-And-Chorus?type=Unofficial
Error downloading https://www.discogs.com/artist/2577229-Rouge-13?type=Unofficial
Error downloading https://www.discogs.com/artist/258129-Jimmy-Lyons-2?type=Unofficial
Error downloading https://www.discogs.com/artist/319329-Napz?type=Unofficial
Error downloading https://www.discogs.com/artist/33229-Leon?type=Unofficial
Error downloading https://www.discogs.com/artist/39329-The-Beat-Club?type=Unofficial
Error downloading https://www.discogs.com/artist/435529-Sleipnir?type=Unofficial
Error downloading https://www.discogs.com/artist/48929-Jeffee?type=Unofficial
Error downloading https://www.discogs.com/artist/62029-Sandman-2?type=Unofficial
Error downloading https://www.discogs.com/artist/731829-Ocarina-2?type=Unofficial
=========================   discogs   =========================
  ===> Found 7620 previous data for ModVal=30
Found 306 artists to get
Error downloading https://www.discogs.com/artist/1248130-Dr-Woggle-The-Radio?type=Unofficial
Error downloading https://www.discogs.com/artist/1468130-Mars-19?type=Unofficial
Error downloading https://www.discogs.com/artist/2561830-Rebel-Drones?type=Unofficial
Error downloading https://www.discogs.com/artist/27330-Visual?type=Unofficial
Error downloading https://www.discogs.com/artist/3030-Manna?type=Unofficial
Error downloading https://www.discogs.com/artist/341030-Glen-Goldsmith?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/30/unofficial/4007830.p (force=False)
  --> This file is 15.3kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/4146830-Ben-Poole-2?type=Unofficial
=========================   discogs   =========================
  ===> Found 7533 previous data for ModVal=31
Found 326 artists to get
Error downloading https://www.discogs.com/artist/1120631-La-Bottega-DellArte?type=Unofficial
Error downloading https://www.discogs.com/artist/131-Dance-Fever?type=Unofficial
Error downloading https://www.discogs.com/artist/136131-Music-Inc?type=Unofficial
Error downloading https://www.discogs.com/artist/1550831-%C3%90%C2%9D%C3%90%C2%B5-%C3%90%C2%98%C3%90%C2%B3%C3%91%C2%80%C3%91%C2%83%C3%91%C2%88%C3%90%C2%BA%C3%90%C2%B8?type=Unofficial
Error downloading https://www.discogs.com/artist/171531-Blacksmoke?type=Unofficial
Error downloading https://www.discogs.com/artist/2012731-Reuben-Sawyer?type=Unofficial
Error downloading https://www.discogs.com/artist/233631-Big-Jim-Slade?type=Unofficial
Error downloading https://www.discogs.com/artist/298931-Joe-Harriott?type=Unofficial
Error downloading https://www.discogs.com/artist/567231-Belinda-Lipscomb?type=Unofficial
Error downloading https://www.discogs.com/artist/7731-L-Double?type=Unofficial
Error downloading https://www.discogs.com/artist/83931-Jay-Strongman?type=Unofficial
=========================   discogs   =========================
  ===> Found 7624 previous data for ModVal=32
Found 345 artists to get
Error downloading https://www.discogs.com/artist/1003232-Dept-Of-Edu?type=Unofficial
Error downloading https://www.discogs.com/artist/203732-Rob-One?type=Unofficial
Error downloading https://www.discogs.com/artist/27832-Nasa?type=Unofficial
Error downloading https://www.discogs.com/artist/5004432-Exizt?type=Unofficial
Error downloading https://www.discogs.com/artist/510332-Bourvil?type=Unofficial
Error downloading https://www.discogs.com/artist/66732-Alva-Noto-Ryuichi-Sakamoto?type=Unofficial
=========================   discogs   =========================
  ===> Found 7602 previous data for ModVal=33
Found 351 artists to get
Error downloading https://www.discogs.com/artist/1295433-Flax-2?type=Unofficial
Error downloading https://www.discogs.com/artist/1633-Ant-Miles?type=Unofficial
Error downloading https://www.discogs.com/artist/37233-Brian-Tony-Gold?type=Unofficial
Error downloading https://www.discogs.com/artist/41233-Jazzy-Jay?type=Unofficial
Error downloading https://www.discogs.com/artist/4376733-Willie-Egans-And-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/447033-Kutmah?type=Unofficial
Error downloading https://www.discogs.com/artist/5674233-Morbid-Mayhem?type=Unofficial
=========================   discogs   =========================
  ===> Found 7592 previous data for ModVal=34
Found 310 artists to get
Error downloading https://www.discogs.com/artist/133634-RMB-and-Friends?type=Unofficial
Error downloading https://www.discogs.com/artist/2031634-Mahsun-K%C4%B1rm%C4%B1z%C4%B1g%C3%BCl?type=Unofficial
Error downloading https://www.discogs.com/artist/353934-Pressure-Release?type=Unofficial
Error downloading https://www.discogs.com/artist/3563234-T%C3%83%C2%A4htiportti-2?type=Unofficial
Error downloading https://www.discogs.com/artist/3727234-Dahna?type=Unofficial
Error downloading https://www.discogs.com/artist/408634-The-Latin-Souls?type=Unofficial
Error downloading https://www.discogs.com/artist/4734-Twisted-Anger?type=Unofficial
Error downloading https://www.discogs.com/artist/5318234-Dj-Gloor?type=Unofficial
Error downloading https://www.discogs.com/artist/559034-Sleep-Clinic?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/34/unofficial/2444134.p (force=False)
  --> This file is 17.1kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7438 previous data for ModVal=35
Found 334 artists to get
Error downloading https://www.discogs.com/artist/108235-Silent-Running?type=Unofficial
Error downloading https://www.discogs.com/artist/1635-Stakka-KTee?type=Unofficial
Error downloading https://www.discogs.com/artist/229035-Final-Edition?type=Unofficial
Error downloading https://www.discogs.com/artist/2652135-Christophe-H%C3%A9ral?type=Unofficial
Error downloading https://www.discogs.com/artist/319935-L%C3%BAcio-Alves?type=Unofficial
Error downloading https://www.discogs.com/artist/378335-Maurizio-Belladonna?type=Unofficial
Error downloading https://www.discogs.com/artist/401635-Carla-Boni?type=Unofficial
Error downloading https://www.discogs.com/artist/4106635-Earforce-2?type=Unofficial
Error downloading https://www.discogs.com/artist/4535-Lucid?type=Unofficial
Error downloading https://www.discogs.com/artist/46235-Global?type=Unofficial
Error downloading https://www.discogs.com/artist/632835-Chiodos?type=Unofficial
Error downloading https://www.discogs.com/artist/7038535-Obscure-Obsolete?type=Unofficial
=========================   discogs   =========================
  ===> Found 7683 previous data for ModVal=36
Found 347 artists to get
Error downloading https://www.discogs.com/artist/102736-Big-Brother?type=Unofficial
Error downloading https://www.discogs.com/artist/12936-Shanie?type=Unofficial
Error downloading https://www.discogs.com/artist/1358536-Stray-8?type=Unofficial
Error downloading https://www.discogs.com/artist/148836-Red-3?type=Unofficial
Error downloading https://www.discogs.com/artist/264336-YUP?type=Unofficial
Error downloading https://www.discogs.com/artist/282436-Bernt-Rosengren?type=Unofficial
Error downloading https://www.discogs.com/artist/29336-DJ-Vitamin-D?type=Unofficial
Error downloading https://www.discogs.com/artist/33936-The-Other-Two?type=Unofficial
Error downloading https://www.discogs.com/artist/4036036-Yunomi?type=Unofficial
Error downloading https://www.discogs.com/artist/575836-Harry-Arnold?type=Unofficial
Error downloading https://www.discogs.com/artist/830636-Gust-Burns?type=Unofficial
Error downloading https://www.discogs.com/artist/92536-Jes-Brieden?type=Unofficial
Error downloading https://www.discogs.com/artist/944636-Mr-Strom?type=Unofficial
Error downloading https://www.discogs.com/artist/99236-Fugitive-2?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/36/unofficial/16336.p (force=False)
  --> This file is 16.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7537 previous data for ModVal=37
Found 332 artists to get
Error downloading https://www.discogs.com/artist/1018437-Moon-Wiring-Club?type=Unofficial
Error downloading https://www.discogs.com/artist/16237-Dakota?type=Unofficial
Error downloading https://www.discogs.com/artist/209437-Lee-Spencer?type=Unofficial
Error downloading https://www.discogs.com/artist/299037-Johnny-Barnes?type=Unofficial
Error downloading https://www.discogs.com/artist/355737-Jonathan-Goldman?type=Unofficial
Error downloading https://www.discogs.com/artist/430437-Mark-Fox?type=Unofficial
Error downloading https://www.discogs.com/artist/48937-Lisahall?type=Unofficial
Error downloading https://www.discogs.com/artist/5443937-Hazel-Mccollum-The-Tren-Teens?type=Unofficial
Error downloading https://www.discogs.com/artist/572337-The-Maskman-And-The-Agents?type=Unofficial
Error downloading https://www.discogs.com/artist/631637-Spectrasoul?type=Unofficial
Error downloading https://www.discogs.com/artist/74937-Grunt?type=Unofficial
=========================   discogs   =========================
  ===> Found 7522 previous data for ModVal=38
Found 310 artists to get
Error downloading https://www.discogs.com/artist/1038138-Social-Disco-Club?type=Unofficial
Error downloading https://www.discogs.com/artist/1198438-Kollegah?type=Unofficial
Error downloading https://www.discogs.com/artist/13738-Numbers?type=Unofficial
Error downloading https://www.discogs.com/artist/3738-SETI?type=Unofficial
Error downloading https://www.discogs.com/artist/41438-M-Factor?type=Unofficial
Error downloading https://www.discogs.com/artist/470538-Matt-Hollywood?type=Unofficial
Error downloading https://www.discogs.com/artist/603338-Roland-Kayn?type=Unofficial
=========================   discogs   =========================
  ===> Found 7537 previous data for ModVal=39
Found 312 artists to get
Error downloading https://www.discogs.com/artist/1144439-The-Ziggens?type=Unofficial
Error downloading https://www.discogs.com/artist/1706439-Chuck-E-Meyers?type=Unofficial
Error downloading https://www.discogs.com/artist/2573939-Andy-Biersack?type=Unofficial
Error downloading https://www.discogs.com/artist/3164639-P%C3%A1l-Esterh%C3%A1zy?type=Unofficial
Error downloading https://www.discogs.com/artist/391039-The-Empires?type=Unofficial
Error downloading https://www.discogs.com/artist/430739-Mike-Waterson?type=Unofficial
Error downloading https://www.discogs.com/artist/5181339-Jimmy-Gilmore-The-Fireballs?type=Unofficial
Error downloading https://www.discogs.com/artist/56639-Skool-Of-Thought?type=Unofficial
Error downloading https://www.discogs.com/artist/839-Deepside?type=Unofficial
=========================   discogs   =========================
  ===> Found 7553 previous data for ModVal=40
Found 343 artists to get
Error downloading https://www.discogs.com/artist/1205840-Future-Islands?type=Unofficial
Error downloading https://www.discogs.com/artist/1453440-Poppis%21?type=Unofficial
Error downloading https://www.discogs.com/artist/1940840-hotliners?type=Unofficial
Error downloading https://www.discogs.com/artist/1961240-Laurent-Naouri?type=Unofficial
Error downloading https://www.discogs.com/artist/280240-Marlo-Hoogstraten?type=Unofficial
Error downloading https://www.discogs.com/artist/37940-Chris-Connelly?type=Unofficial
Error downloading https://www.discogs.com/artist/639840-Franz-Lehrndorfer?type=Unofficial
Error downloading https://www.discogs.com/artist/766540-Ruffneck-Featuring-Yavahn?type=Unofficial
Error downloading https://www.discogs.com/artist/82540-Super-3?type=Unofficial
Error downloading https://www.discogs.com/artist/838040-Kammerorchester-Berlin?type=Unofficial
Error downloading https://www.discogs.com/artist/8540-Jerry-Ropero?type=Unofficial
=========================   discogs   =========================
  ===> Found 7759 previous data for ModVal=41
Found 349 artists to get
Error downloading https://www.discogs.com/artist/1376041-Dj-Evo?type=Unofficial
Error downloading https://www.discogs.com/artist/1641-Fortran?type=Unofficial
Error downloading https://www.discogs.com/artist/184441-Bernard-Heidsieck?type=Unofficial
Error downloading https://www.discogs.com/artist/2087041-Judas-9?type=Unofficial
Error downloading https://www.discogs.com/artist/2147641-DJ-Guilty?type=Unofficial
Error downloading https://www.discogs.com/artist/2463841-Sacred-Heart?type=Unofficial
Error downloading https://www.discogs.com/artist/2469341-Creation-Of-Sunlight?type=Unofficial
Error downloading https://www.discogs.com/artist/2644641-Teddy-McRae-Orchestra?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/41/unofficial/351541.p (force=False)
  --> This file is 26.5kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/357441-Eyedea-Abilities?type=Unofficial
Error downloading https://www.discogs.com/artist/611741-Freddie-Gorman?type=Unofficial
Error downloading https://www.discogs.com/artist/751941-Erno-Rapee?type=Unofficial
=========================   discogs   =========================
  ===> Found 7607 previous data for ModVal=42
Found 314 artists to get
Error downloading https://www.discogs.com/artist/1198742-Dennis-Day?type=Unofficial
Error downloading https://www.discogs.com/artist/1395642-Frivolvol?type=Unofficial
Error downloading https://www.discogs.com/artist/1835542-Dj-Diggz?type=Unofficial
Error downloading https://www.discogs.com/artist/257142-The-Flesh-Eaters?type=Unofficial
Error downloading https://www.discogs.com/artist/292442-Justin-Hinds?type=Unofficial
Error downloading https://www.discogs.com/artist/36142-Vernessa-Mitchell?type=Unofficial
Error downloading https://www.discogs.com/artist/429942-Bone-Symphony?type=Unofficial
Error downloading https://www.discogs.com/artist/529142-Bosse-Skoglund?type=Unofficial
Error downloading https://www.discogs.com/artist/5423542-Twin-Sisters-Productions-Inc?type=Unofficial
Error downloading https://www.discogs.com/artist/5442-Casino-Versus-Japan?type=Unofficial
Error downloading https://www.discogs.com/artist/703742-Rod-Morris-2?type=Unofficial
Error downloading https://www.discogs.com/artist/790742-Tom-Reeves?type=Unofficial
Error downloading https://www.discogs.com/artist/971242-Calcutta?type=Unofficial
=========================   discogs   =========================
  ===> Found 7622 previous data for ModVal=43
Found 339 artists to get
Error downloading https://www.discogs.com/artist/1046943-Hamburger-Kammerorchester?type=Unofficial
Error downloading https://www.discogs.com/artist/3085043-DJ-Shame-2?type=Unofficial
Error downloading https://www.discogs.com/artist/499643-Fausto-Rossi?type=Unofficial
Error downloading https://www.discogs.com/artist/648343-Al-Smith?type=Unofficial
Error downloading https://www.discogs.com/artist/6956543-Hideamans-Way?type=Unofficial
Error downloading https://www.discogs.com/artist/83143-Pebbles?type=Unofficial
=========================   discogs   =========================
  ===> Found 7453 previous data for ModVal=44
Found 345 artists to get
Error downloading https://www.discogs.com/artist/1104044-Have-A-Nice-Life?type=Unofficial
Error downloading https://www.discogs.com/artist/270544-Danny-Janssen?type=Unofficial
Error downloading https://www.discogs.com/artist/45644-John-Wesley-Harding?type=Unofficial
Error downloading https://www.discogs.com/artist/748944-Jump-With-Joey?type=Unofficial
=========================   discogs   =========================
  ===> Found 7416 previous data for ModVal=45
Found 337 artists to get
Error downloading https://www.discogs.com/artist/111545-DJ-Jeroenski?type=Unofficial
Error downloading https://www.discogs.com/artist/1439545-Kamal-Ahmed?type=Unofficial
Error downloading https://www.discogs.com/artist/190445-Jean-Louis-Aubert?type=Unofficial
Error downloading https://www.discogs.com/artist/2023545-Orchester-Carl-Woitschach?type=Unofficial
Error downloading https://www.discogs.com/artist/252045-Jill-Sobule?type=Unofficial
Error downloading https://www.discogs.com/artist/2891145-Ewan-Dobson?type=Unofficial
Error downloading https://www.discogs.com/artist/293445-Cynthia-Harrell?type=Unofficial
Error downloading https://www.discogs.com/artist/34945-Madoka?type=Unofficial
=========================   discogs   =========================
  ===> Found 7493 previous data for ModVal=46
Found 328 artists to get
Error downloading https://www.discogs.com/artist/10046-DJ-Pulse?type=Unofficial
Error downloading https://www.discogs.com/artist/104746-Count-Bass-D?type=Unofficial
Error downloading https://www.discogs.com/artist/1102746-Los-Monaguillosh?type=Unofficial
Error downloading https://www.discogs.com/artist/122446-Daniel-Ash?type=Unofficial
Error downloading https://www.discogs.com/artist/1788046-%CE%97%CE%BB%CE%AF%CE%B1%CF%82-%CE%91%CF%83%CE%B2%CE%B5%CF%83%CF%84%CF%8C%CF%80%CE%BF%CF%85%CE%BB%CE%BF%CF%82?type=Unofficial
Error downloading https://www.discogs.com/artist/219746-Street-Beat?type=Unofficial
Error downloading https://www.discogs.com/artist/263346-Nikki-Sudden?type=Unofficial
Error downloading https://www.discogs.com/artist/268846-The-Stones?type=Unofficial
Error downloading https://www.discogs.com/artist/295346-Trevor-Lucas?type=Unofficial
Error downloading https://www.discogs.com/artist/332446-Basto%21?type=Unofficial
Error downloading https://www.discogs.com/artist/381846-Mannish?type=Unofficial
Error downloading https://www.discogs.com/artist/59946-Loscil?type=Unofficial
Error downloading https://www.discogs.com/artist/64546-Funny-F?type=Unofficial
Error downloading https://www.discogs.com/artist/7810846-Notorious-BIG-50-Cent?type=Unofficial
Error downloading https://www.discogs.com/artist/8046-Marlow?type=Unofficial
Error downloading https://www.discogs.com/artist/94146-Too-Strong?type=Unofficial
=========================   discogs   =========================
  ===> Found 7413 previous data for ModVal=47
Found 320 artists to get
Error downloading https://www.discogs.com/artist/1508547-2000-Mix?type=Unofficial
Error downloading https://www.discogs.com/artist/161447-DJ-Ryow?type=Unofficial
Error downloading https://www.discogs.com/artist/2193747-Deep-Disco?type=Unofficial
Error downloading https://www.discogs.com/artist/2405647-Ultrademon?type=Unofficial
Error downloading https://www.discogs.com/artist/278347-Frank-N-Dank?type=Unofficial
Error downloading https://www.discogs.com/artist/341947-David-Campbell?type=Unofficial
Error downloading https://www.discogs.com/artist/371747-Rejuvenate?type=Unofficial
Error downloading https://www.discogs.com/artist/701547-Cozi?type=Unofficial
Error downloading https://www.discogs.com/artist/734847-Garrett-Klahn?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/47/unofficial/1685347.p (force=False)
  --> This file is 17.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7706 previous data for ModVal=48
Found 308 artists to get
Error downloading https://www.discogs.com/artist/16248-Stranger?type=Unofficial
Error downloading https://www.discogs.com/artist/2565548-Girls-With-Guitars?type=Unofficial
Error downloading https://www.discogs.com/artist/2722848-Various-Godfather-EP?type=Unofficial
Error downloading https://www.discogs.com/artist/2850248-The-Strand-6?type=Unofficial
Error downloading https://www.discogs.com/artist/3448-Lab-Rats?type=Unofficial
Error downloading https://www.discogs.com/artist/4668248-Kis-Grof%C3%83%C2%B3?type=Unofficial
Error downloading https://www.discogs.com/artist/696048-The-Superlatives-3?type=Unofficial
=========================   discogs   =========================
  ===> Found 7655 previous data for ModVal=49
Found 313 artists to get
Error downloading https://www.discogs.com/artist/110949-Punish-Yourself?type=Unofficial
Error downloading https://www.discogs.com/artist/1247149-The-Pride-3?type=Unofficial
Error downloading https://www.discogs.com/artist/131049-Paddy-Kingsland?type=Unofficial
Error downloading https://www.discogs.com/artist/272449-Cure?type=Unofficial
Error downloading https://www.discogs.com/artist/761549-Svart-Framtid?type=Unofficial
Error downloading https://www.discogs.com/artist/9949-DJ-Technics?type=Unofficial
=========================   discogs   =========================
  ===> Found 7604 previous data for ModVal=50
Found 332 artists to get
Error downloading https://www.discogs.com/artist/201450-Solitude-FX?type=Unofficial
Error downloading https://www.discogs.com/artist/2559550-Yuu?type=Unofficial
Error downloading https://www.discogs.com/artist/27650-Reefer-Decree?type=Unofficial
Error downloading https://www.discogs.com/artist/332450-Jimmy-Lewis-2?type=Unofficial
Error downloading https://www.discogs.com/artist/371750-108-3?type=Unofficial
Error downloading https://www.discogs.com/artist/774750-Inoki?type=Unofficial
=========================   discogs   =========================
  ===> Found 7678 previous data for ModVal=51
Found 324 artists to get
Error downloading https://www.discogs.com/artist/181351-GLC?type=Unofficial
Error downloading https://www.discogs.com/artist/2045151-Kahn-5?type=Unofficial
Error downloading https://www.discogs.com/artist/2055951-John-Mellencamp?type=Unofficial
Error downloading https://www.discogs.com/artist/243951-Bobby-Franklins-Insanity?type=Unofficial
Error downloading https://www.discogs.com/artist/325951-Mask-5?type=Unofficial
Error downloading https://www.discogs.com/artist/505451-Nino-Oliviero?type=Unofficial
Error downloading https://www.discogs.com/artist/5348051-Bruce-Springsteen-Bon-Jovi?type=Unofficial
Error downloading https://www.discogs.com/artist/651-Digital?type=Unofficial
Error downloading https://www.discogs.com/artist/7451-Cusp?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/51/unofficial/1234651.p (force=False)
  --> This file is 17.1kB.
Done. Sleeping for 2 seconds
Saving /Volumes/Piggy/Discog/artists-discogs/51/unofficial/5223351.p (force=False)
  --> This file is 15.3kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7682 previous data for ModVal=52
Found 372 artists to get
Error downloading https://www.discogs.com/artist/1054252-Franco-Califano?type=Unofficial
Error downloading https://www.discogs.com/artist/124052-Maxim-Sinclair?type=Unofficial
Error downloading https://www.discogs.com/artist/430752-Lal-Waterson?type=Unofficial
Error downloading https://www.discogs.com/artist/6346252-Herbie-Hancock-headhunters?type=Unofficial
Error downloading https://www.discogs.com/artist/665552-Various-3?type=Unofficial
Error downloading https://www.discogs.com/artist/962052-Skirt?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/52/unofficial/667752.p (force=False)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7651 previous data for ModVal=53
Found 331 artists to get
Error downloading https://www.discogs.com/artist/124053-Paul-Benjamin?type=Unofficial
Error downloading https://www.discogs.com/artist/2853-Kemal-Rob-Data?type=Unofficial
Error downloading https://www.discogs.com/artist/292153-The-Extras?type=Unofficial
Error downloading https://www.discogs.com/artist/349953-Prinz-Porno?type=Unofficial
Error downloading https://www.discogs.com/artist/3622953-Richard-Moreland?type=Unofficial
Error downloading https://www.discogs.com/artist/6360753-Ruth-Brown-And-Her-Rhythmakers-With-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/707253-Walter-Rhodes?type=Unofficial
Error downloading https://www.discogs.com/artist/856653-Schubert-3?type=Unofficial
=========================   discogs   =========================
  ===> Found 7727 previous data for ModVal=54
Found 319 artists to get
Error downloading https://www.discogs.com/artist/1107254-Gnod?type=Unofficial
Error downloading https://www.discogs.com/artist/150554-Guilty-Simpson?type=Unofficial
Error downloading https://www.discogs.com/artist/1621054-DJ-NikOne?type=Unofficial
Error downloading https://www.discogs.com/artist/2109254-Mission-Infect?type=Unofficial
Error downloading https://www.discogs.com/artist/2854-Stakka-Skynet?type=Unofficial
Error downloading https://www.discogs.com/artist/3371254-July-Talk?type=Unofficial
Error downloading https://www.discogs.com/artist/4332854-JJBurner?type=Unofficial
Error downloading https://www.discogs.com/artist/446154-Albert-Kuvezin?type=Unofficial
Error downloading https://www.discogs.com/artist/4954-Ryme-Tyme?type=Unofficial
=========================   discogs   =========================
  ===> Found 7677 previous data for ModVal=55
Found 333 artists to get
Error downloading https://www.discogs.com/artist/1028055-Tramteatret?type=Unofficial
Error downloading https://www.discogs.com/artist/122055-Point-Blank-3?type=Unofficial
Error downloading https://www.discogs.com/artist/2211255-TEST-9?type=Unofficial
Error downloading https://www.discogs.com/artist/2779455-Soulspell?type=Unofficial
Error downloading https://www.discogs.com/artist/2790255-BEP?type=Unofficial
Error downloading https://www.discogs.com/artist/296255-Jimmy-Destri?type=Unofficial
Error downloading https://www.discogs.com/artist/3555-Negrocan?type=Unofficial
Error downloading https://www.discogs.com/artist/404655-Mistah-FAB?type=Unofficial
Error downloading https://www.discogs.com/artist/430555-The-Sperm?type=Unofficial
Error downloading https://www.discogs.com/artist/446155-Yat-Kha?type=Unofficial
Error downloading https://www.discogs.com/artist/69955-Dover?type=Unofficial
Error downloading https://www.discogs.com/artist/90855-Heliotrope?type=Unofficial
=========================   discogs   =========================
  ===> Found 7620 previous data for ModVal=56
Found 354 artists to get
Error downloading https://www.discogs.com/artist/1220456-Riff-3?type=Unofficial
Error downloading https://www.discogs.com/artist/2223556-Academics-2?type=Unofficial
Error downloading https://www.discogs.com/artist/2642356-Dr-Livingstone-2?type=Unofficial
Error downloading https://www.discogs.com/artist/306056-Normil-Hawaiians?type=Unofficial
Error downloading https://www.discogs.com/artist/374856-Quarterflash?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/56/unofficial/3900856.p (force=False)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/560556-Elijah-Williams-Jr?type=Unofficial
Error downloading https://www.discogs.com/artist/818356-Primary-1?type=Unofficial
Error downloading https://www.discogs.com/artist/858756-The-Royal-Stockholm-Philharmonic-Orchestra?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/56/unofficial/264456.p (force=False)
  --> This file is 17.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7600 previous data for ModVal=57
Found 347 artists to get
Error downloading https://www.discogs.com/artist/1634557-Lawson-Haggart-Jazz-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/266857-Chokehold?type=Unofficial
Error downloading https://www.discogs.com/artist/267857-Patty-Smyth?type=Unofficial
Error downloading https://www.discogs.com/artist/434457-Preacher-3?type=Unofficial
Error downloading https://www.discogs.com/artist/4493057-Ian-Gillan-And-The-Javelins?type=Unofficial
Error downloading https://www.discogs.com/artist/4982757-Theo-Shanab?type=Unofficial
Error downloading https://www.discogs.com/artist/6312757-Ruth-Brown-With-Orchestra?type=Unofficial
=========================   discogs   =========================
  ===> Found 7542 previous data for ModVal=58
Found 318 artists to get
Error downloading https://www.discogs.com/artist/12758-London-Fiesta?type=Unofficial
Error downloading https://www.discogs.com/artist/1400458-The-Honeydreamers?type=Unofficial
Error downloading https://www.discogs.com/artist/148058-Exile-6?type=Unofficial
Error downloading https://www.discogs.com/artist/2312258-DJ-Quest-6?type=Unofficial
Error downloading https://www.discogs.com/artist/2356058-The-First-National-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/249758-Ziggy-Marley?type=Unofficial
Error downloading https://www.discogs.com/artist/27758-4-Clubbers?type=Unofficial
Error downloading https://www.discogs.com/artist/29358-Suzanne-Palmer?type=Unofficial
Error downloading https://www.discogs.com/artist/3343158-Jorge-Cardoso-2?type=Unofficial
Error downloading https://www.discogs.com/artist/655958-DJ-Basilisk?type=Unofficial
=========================   discogs   =========================
  ===> Found 7732 previous data for ModVal=59
Found 304 artists to get
Error downloading https://www.discogs.com/artist/133959-Tic-Tac-Toe?type=Unofficial
Error downloading https://www.discogs.com/artist/13559-Elements?type=Unofficial
Error downloading https://www.discogs.com/artist/1742559-IA-Batiste?type=Unofficial
Error downloading https://www.discogs.com/artist/394459-Ismael-Quintana?type=Unofficial
Error downloading https://www.discogs.com/artist/4200559-American-Pulverizer?type=Unofficial
Error downloading https://www.discogs.com/artist/700759-The-Spits?type=Unofficial
Error downloading https://www.discogs.com/artist/799359-Miles-Kane?type=Unofficial
Error downloading https://www.discogs.com/artist/843259-Luis-Cilia?type=Unofficial
Error downloading https://www.discogs.com/artist/996059-Pete-Bonus?type=Unofficial
=========================   discogs   =========================
  ===> Found 7656 previous data for ModVal=60
Found 335 artists to get
Error downloading https://www.discogs.com/artist/1166760-Alonzo-Scales?type=Unofficial
Error downloading https://www.discogs.com/artist/2333160-Chorus-And-Orchestra-Of-The-Teatro-Di-San-Carlo-Di-Napoli?type=Unofficial
Error downloading https://www.discogs.com/artist/240660-Teej?type=Unofficial
Error downloading https://www.discogs.com/artist/348860-IQ-3?type=Unofficial
Error downloading https://www.discogs.com/artist/5521260-Doukutsu-Monogatari?type=Unofficial
Error downloading https://www.discogs.com/artist/669060-Bride?type=Unofficial
=========================   discogs   =========================
  ===> Found 7595 previous data for ModVal=61
Found 325 artists to get
Error downloading https://www.discogs.com/artist/1851061-Lil-Ceasar-2?type=Unofficial
Error downloading https://www.discogs.com/artist/287861-Tony-Garnier?type=Unofficial
Error downloading https://www.discogs.com/artist/377361-The-Finn-Brothers?type=Unofficial
Error downloading https://www.discogs.com/artist/429961-The-Jacobites?type=Unofficial
Error downloading https://www.discogs.com/artist/4488061-The-Bittersweets-5?type=Unofficial
=========================   discogs   =========================
  ===> Found 7711 previous data for ModVal=62
Found 345 artists to get
Error downloading https://www.discogs.com/artist/11762-The-Dronez?type=Unofficial
Error downloading https://www.discogs.com/artist/1551762-The-Live-Experience-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/165262-Pont-Aeri?type=Unofficial
Error downloading https://www.discogs.com/artist/200962-Animals?type=Unofficial
Error downloading https://www.discogs.com/artist/20562-Reggie-Hall?type=Unofficial
Error downloading https://www.discogs.com/artist/23762-Gabi-Delgado?type=Unofficial
Error downloading https://www.discogs.com/artist/334662-Caustic?type=Unofficial
Error downloading https://www.discogs.com/artist/369762-Doomsday-Productions?type=Unofficial
Error downloading https://www.discogs.com/artist/3830862-Wenge-Musica-BCBG?type=Unofficial
Error downloading https://www.discogs.com/artist/488362-Giacomo-Meyerbeer?type=Unofficial
Error downloading https://www.discogs.com/artist/665162-Scythe?type=Unofficial
Error downloading https://www.discogs.com/artist/6662-DJ-Reality?type=Unofficial
Error downloading https://www.discogs.com/artist/863562-Bread-And-Water-2?type=Unofficial
=========================   discogs   =========================
  ===> Found 7701 previous data for ModVal=63
Found 355 artists to get
Error downloading https://www.discogs.com/artist/1477463-Bedlam-7?type=Unofficial
Error downloading https://www.discogs.com/artist/1809563-The-Don-Harrison-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/233863-Private-Lives?type=Unofficial
Error downloading https://www.discogs.com/artist/2595863-Lucky-Carmichael?type=Unofficial
Error downloading https://www.discogs.com/artist/267063-Heaven-Shall-Burn?type=Unofficial
Error downloading https://www.discogs.com/artist/28063-Fred-Jorio?type=Unofficial
Error downloading https://www.discogs.com/artist/3478363-Tuva-2?type=Unofficial
Error downloading https://www.discogs.com/artist/355463-Recover?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/63/unofficial/3568763.p (force=False)
  --> This file is 24.2kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/499363-Alexander-Spence?type=Unofficial
Error downloading https://www.discogs.com/artist/531563-DJ-%D0%9A%D0%B8%D1%80%D0%B8%D0%BB%D0%BEff?type=Unofficial
Error downloading https://www.discogs.com/artist/61563-Hard-Trance?type=Unofficial
Error downloading https://www.discogs.com/artist/727663-Shigeru-Matsuzaki?type=Unofficial
Error downloading https://www.discogs.com/artist/843463-The-Waikikis?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/63/unofficial/1499663.p (force=False)
  --> This file is 17.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7757 previous data for ModVal=64
Found 338 artists to get
Error downloading https://www.discogs.com/artist/1088064-Dana-Cooper?type=Unofficial
Error downloading https://www.discogs.com/artist/1448764-%CE%A7%CE%B1%CE%BF%CF%84%CE%B9%CE%BA%CF%8C-%CE%A4%CE%AD%CE%BB%CE%BF%CF%82?type=Unofficial
Error downloading https://www.discogs.com/artist/339764-Potato-5?type=Unofficial
Error downloading https://www.discogs.com/artist/3450364-Young-Pone?type=Unofficial
Error downloading https://www.discogs.com/artist/512764-Angela-Strehli?type=Unofficial
Error downloading https://www.discogs.com/artist/5229464-Guys-Dolls-5?type=Unofficial
Error downloading https://www.discogs.com/artist/7464-DJs-Unite?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/64/unofficial/2991664.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
Saving /Volumes/Piggy/Discog/artists-discogs/64/unofficial/3453464.p (force=False)
  --> This file is 16.9kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7680 previous data for ModVal=65
Found 296 artists to get
Error downloading https://www.discogs.com/artist/10465-Mask?type=Unofficial
Error downloading https://www.discogs.com/artist/1087265-Ralph-Marterie?type=Unofficial
Error downloading https://www.discogs.com/artist/247365-Ron-Richardson?type=Unofficial
Error downloading https://www.discogs.com/artist/254865-The-Rhythm-Rockers?type=Unofficial
Error downloading https://www.discogs.com/artist/264865-Alana-Dante?type=Unofficial
Error downloading https://www.discogs.com/artist/282265-Autonomous-Zone?type=Unofficial
Error downloading https://www.discogs.com/artist/2835565-Jimmy-Ricks-The-Ravens?type=Unofficial
Error downloading https://www.discogs.com/artist/4013265-Johnny-Mastro-And-The-Mamas-Boys?type=Unofficial
Error downloading https://www.discogs.com/artist/4265-Dean-Coleman?type=Unofficial
Error downloading https://www.discogs.com/artist/4266465-Jake-Chudnow?type=Unofficial
Error downloading https://www.discogs.com/artist/54965-NDX?type=Unofficial
=========================   discogs   =========================
  ===> Found 7632 previous data for ModVal=66
Found 325 artists to get
Error downloading https://www.discogs.com/artist/10466-Swabe?type=Unofficial
Error downloading https://www.discogs.com/artist/1106266-Agust%C3%ADn-Barrios-Mangor%C3%A9?type=Unofficial
Error downloading https://www.discogs.com/artist/147666-Yana-Kay?type=Unofficial
Error downloading https://www.discogs.com/artist/4318566-Pearl-Handle-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/5070666-Castiel-2?type=Unofficial
=========================   discogs   =========================
  ===> Found 7728 previous data for ModVal=67
Found 359 artists to get
Error downloading https://www.discogs.com/artist/15467-CoH?type=Unofficial
Error downloading https://www.discogs.com/artist/20367-Legacy?type=Unofficial
Error downloading https://www.discogs.com/artist/212667-NgaFsh?type=Unofficial
Error downloading https://www.discogs.com/artist/216667-Mean-Green?type=Unofficial
Error downloading https://www.discogs.com/artist/263767-Beyond-Description?type=Unofficial
Error downloading https://www.discogs.com/artist/34067-Pete-Hammond?type=Unofficial
Error downloading https://www.discogs.com/artist/630867-William-Bell-2?type=Unofficial
Error downloading https://www.discogs.com/artist/648667-Sylvia-7?type=Unofficial
Error downloading https://www.discogs.com/artist/654667-The-LoveCrave?type=Unofficial
Error downloading https://www.discogs.com/artist/70467-Xorcist?type=Unofficial
Error downloading https://www.discogs.com/artist/887967-Ray-Scott-2?type=Unofficial
Error downloading https://www.discogs.com/artist/950167-Rich-Stearns?type=Unofficial
Error downloading https://www.discogs.com/artist/99567-Toms-Project?type=Unofficial
=========================   discogs   =========================
  ===> Found 7474 previous data for ModVal=68
Found 320 artists to get
Error downloading https://www.discogs.com/artist/1056568-Zagreba%C4%8Dki-Solisti?type=Unofficial
Error downloading https://www.discogs.com/artist/1059168-Mari-Nakamoto?type=Unofficial
Error downloading https://www.discogs.com/artist/134068-Distant-Drums?type=Unofficial
Error downloading https://www.discogs.com/artist/2505468-Giorgio-Canali-Rosso-Fuoco?type=Unofficial
Error downloading https://www.discogs.com/artist/3161968-Ivory-Tiger?type=Unofficial
Error downloading https://www.discogs.com/artist/689168-Robert-Byrd?type=Unofficial
Error downloading https://www.discogs.com/artist/78668-The-Deele?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/68/unofficial/53868.p (force=False)
  --> This file is 17.9kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7618 previous data for ModVal=69
Found 329 artists to get
Error downloading https://www.discogs.com/artist/2195669-Rocky-Campbell?type=Unofficial
Error downloading https://www.discogs.com/artist/2681469-Icewind?type=Unofficial
Error downloading https://www.discogs.com/artist/278269-XMen?type=Unofficial
Error downloading https://www.discogs.com/artist/280269-Sean-Nelson?type=Unofficial
Error downloading https://www.discogs.com/artist/430769-Cheo-Feliciano?type=Unofficial
Error downloading https://www.discogs.com/artist/818569-Les-Baxter-His-Chorus-And-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/850869-Fly-Bi-Night?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/69/unofficial/360369.p (force=False)
  --> This file is 17.2kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7536 previous data for ModVal=70
Found 366 artists to get
Error downloading https://www.discogs.com/artist/13870-The-B-Boys?type=Unofficial
Error downloading https://www.discogs.com/artist/2442770-%CE%A3%CE%B1%CE%BC%CF%80%CF%81%CE%AF%CE%BD%CE%B1?type=Unofficial
Error downloading https://www.discogs.com/artist/271770-The-Macc-Lads?type=Unofficial
Error downloading https://www.discogs.com/artist/3773570-Diamond-Ortiz?type=Unofficial
Error downloading https://www.discogs.com/artist/5775970-Elton-John-Billy-Joel?type=Unofficial
Error downloading https://www.discogs.com/artist/599270-Roy-Montrell?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/70/unofficial/4249570.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7573 previous data for ModVal=71
Found 329 artists to get
Error downloading https://www.discogs.com/artist/1017371-Ebenezer-Obey?type=Unofficial
Error downloading https://www.discogs.com/artist/1271-Tranquility-Bass?type=Unofficial
Error downloading https://www.discogs.com/artist/1318471-Yosuke-Onuma?type=Unofficial
Error downloading https://www.discogs.com/artist/1718771-Yellow-Brick-Road?type=Unofficial
Error downloading https://www.discogs.com/artist/191271-Biddu?type=Unofficial
Error downloading https://www.discogs.com/artist/2734971-Las-Antorchas?type=Unofficial
Error downloading https://www.discogs.com/artist/5588471-Dr-Mario-5?type=Unofficial
Error downloading https://www.discogs.com/artist/6171-Facs?type=Unofficial
Error downloading https://www.discogs.com/artist/770571-Dr-Demento?type=Unofficial
Error downloading https://www.discogs.com/artist/904171-Yumi-Matsuzawa?type=Unofficial
=========================   discogs   =========================
  ===> Found 7585 previous data for ModVal=72
Found 322 artists to get
Error downloading https://www.discogs.com/artist/314972-The-Lincoln-Center-Jazz-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/4235572-Nozomu-Wakais-Destinia?type=Unofficial
Error downloading https://www.discogs.com/artist/5335372-Eri-Lime?type=Unofficial
=========================   discogs   =========================
  ===> Found 7598 previous data for ModVal=73
Found 353 artists to get
Error downloading https://www.discogs.com/artist/1291373-Loka-Nunda?type=Unofficial
Error downloading https://www.discogs.com/artist/1432073-Deceptor-2?type=Unofficial
Error downloading https://www.discogs.com/artist/1600173-Kreton?type=Unofficial
Error downloading https://www.discogs.com/artist/181973-Avalon-4?type=Unofficial
Error downloading https://www.discogs.com/artist/205073-Federation?type=Unofficial
Error downloading https://www.discogs.com/artist/217273-Juhani?type=Unofficial
Error downloading https://www.discogs.com/artist/2403173-Falling-In-Reverse?type=Unofficial
Error downloading https://www.discogs.com/artist/242973-Seiei-Jack?type=Unofficial
Error downloading https://www.discogs.com/artist/416073-New-York-Scratch-Masters?type=Unofficial
Error downloading https://www.discogs.com/artist/450273-?type=Unofficial
Error downloading https://www.discogs.com/artist/84073-Pink-Industry?type=Unofficial
=========================   discogs   =========================
  ===> Found 7651 previous data for ModVal=74
Found 343 artists to get
Error downloading https://www.discogs.com/artist/1111674-John-Berberian?type=Unofficial
Error downloading https://www.discogs.com/artist/1813674-Chuck-U?type=Unofficial
Error downloading https://www.discogs.com/artist/2046974-Hydropneumothorax?type=Unofficial
Error downloading https://www.discogs.com/artist/2645574-Coro-E-Orchestra-DellAccademia-Di-Santa-Cecilia?type=Unofficial
Error downloading https://www.discogs.com/artist/2748774-Yarmulke?type=Unofficial
Error downloading https://www.discogs.com/artist/2948474-TT-34?type=Unofficial
Error downloading https://www.discogs.com/artist/338774-Mike-Joyce?type=Unofficial
Error downloading https://www.discogs.com/artist/341574-Homesick-James?type=Unofficial
Error downloading https://www.discogs.com/artist/4107374-Julian-Wareing?type=Unofficial
Error downloading https://www.discogs.com/artist/529074-The-Sinceros?type=Unofficial
Error downloading https://www.discogs.com/artist/58874-Apollo-XI?type=Unofficial
Error downloading https://www.discogs.com/artist/7374-Wishdokta?type=Unofficial
=========================   discogs   =========================
  ===> Found 7631 previous data for ModVal=75
Found 336 artists to get
Error downloading https://www.discogs.com/artist/1333275-Metz?type=Unofficial
Error downloading https://www.discogs.com/artist/2075-Cloud-9?type=Unofficial
Error downloading https://www.discogs.com/artist/252475-The-Sparkles?type=Unofficial
Error downloading https://www.discogs.com/artist/258175-Mr-Ed-Jumps-The-Gun?type=Unofficial
Error downloading https://www.discogs.com/artist/290775-Marsheaux?type=Unofficial
Error downloading https://www.discogs.com/artist/359475-Henry-Lee-Summer?type=Unofficial
Error downloading https://www.discogs.com/artist/5336975-Capolinea-Internazional-Funk-Banda?type=Unofficial
Error downloading https://www.discogs.com/artist/5420475-The-Sleigh-Ride-Singers?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/75/unofficial/5691275.p (force=False)
  --> This file is 17.5kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7589 previous data for ModVal=76
Found 344 artists to get
Error downloading https://www.discogs.com/artist/10576-DJ-Noize?type=Unofficial
Error downloading https://www.discogs.com/artist/1085676-Magic-Muscle?type=Unofficial
Error downloading https://www.discogs.com/artist/1756776-Napol%C3%A9on-Coste?type=Unofficial
Error downloading https://www.discogs.com/artist/255376-Bobby-Whitlock?type=Unofficial
Error downloading https://www.discogs.com/artist/3326876-The-Joystix?type=Unofficial
Error downloading https://www.discogs.com/artist/3947676-Shawn-Sahm-And-Prezence?type=Unofficial
Error downloading https://www.discogs.com/artist/510576-Edgar-Winters-White-Trash?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/76/unofficial/4264976.p (force=False)
  --> This file is 17.0kB.
Done. Sleeping for 2 seconds
Saving /Volumes/Piggy/Discog/artists-discogs/76/unofficial/1033576.p (force=False)
  --> This file is 16.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7600 previous data for ModVal=77
Found 341 artists to get
Error downloading https://www.discogs.com/artist/1245777-Britten-Sinfonia?type=Unofficial
Error downloading https://www.discogs.com/artist/14577-Thomas-Penton?type=Unofficial
Error downloading https://www.discogs.com/artist/152277-BabyFace-Jay?type=Unofficial
Error downloading https://www.discogs.com/artist/1558977-Mushrooms-Project?type=Unofficial
Error downloading https://www.discogs.com/artist/1606377-Rock-Candy-5?type=Unofficial
Error downloading https://www.discogs.com/artist/2063277-C-Arjun?type=Unofficial
Error downloading https://www.discogs.com/artist/2129077-Infernal-Curse?type=Unofficial
Error downloading https://www.discogs.com/artist/2140877-Liberators?type=Unofficial
Error downloading https://www.discogs.com/artist/262277-Ann-Rabson?type=Unofficial
Error downloading https://www.discogs.com/artist/314877-Morning-Musume?type=Unofficial
Error downloading https://www.discogs.com/artist/345277-DJ-Renat?type=Unofficial
Error downloading https://www.discogs.com/artist/5202377-Big-Duke-4?type=Unofficial
Error downloading https://www.discogs.com/artist/828377-Koroded?type=Unofficial
=========================   discogs   =========================
  ===> Found 7749 previous data for ModVal=78
Found 339 artists to get
Error downloading https://www.discogs.com/artist/112478-Demolition?type=Unofficial
Error downloading https://www.discogs.com/artist/131878-Gene-Dunlap?type=Unofficial
Error downloading https://www.discogs.com/artist/153078-Scottie?type=Unofficial
Error downloading https://www.discogs.com/artist/1752278-Raul-Porchetto?type=Unofficial
Error downloading https://www.discogs.com/artist/2409178-B-Jam-2?type=Unofficial
Error downloading https://www.discogs.com/artist/2601278-Hristo-2?type=Unofficial
Error downloading https://www.discogs.com/artist/389478-The-Highlighters-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/39878-Alice-D-In-Wonderland?type=Unofficial
Error downloading https://www.discogs.com/artist/5081978-Eric-Bibb-And-North-Country-Far-With-Danny-Thompson?type=Unofficial
=========================   discogs   =========================
  ===> Found 7629 previous data for ModVal=79
Found 324 artists to get
Error downloading https://www.discogs.com/artist/1036479-Race-War?type=Unofficial
Error downloading https://www.discogs.com/artist/168579-Nico-4?type=Unofficial
Error downloading https://www.discogs.com/artist/3768579-Djemeia?type=Unofficial
Error downloading https://www.discogs.com/artist/4545579-Fox-McGee?type=Unofficial
Error downloading https://www.discogs.com/artist/876179-LAngelo-Misterioso?type=Unofficial
Error downloading https://www.discogs.com/artist/5945379-Chris-Nisley?type=Unofficial
=========================   discogs   =========================
  ===> Found 7654 previous data for ModVal=80
Found 318 artists to get
Error downloading https://www.discogs.com/artist/1648180-Bad-B-%D0%90%D0%BB%D1%8C%D1%8F%D0%BD%D1%81?type=Unofficial
Error downloading https://www.discogs.com/artist/171780-Jamaaladeen-Tacuma?type=Unofficial
Error downloading https://www.discogs.com/artist/280-Boymerang?type=Unofficial
Error downloading https://www.discogs.com/artist/307580-Norio-Maeda?type=Unofficial
Error downloading https://www.discogs.com/artist/374780-Marc-Mac?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/80/unofficial/4464180.p (force=False)
  --> This file is 15.7kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7789 previous data for ModVal=81
Found 353 artists to get
Error downloading https://www.discogs.com/artist/1213281-Merle-Haggard-And-The-Strangers?type=Unofficial
Error downloading https://www.discogs.com/artist/1425381-New-Tango-Orquestra?type=Unofficial
Error downloading https://www.discogs.com/artist/1446581-2002-GR?type=Unofficial
Error downloading https://www.discogs.com/artist/1555581-Kriek-Balise?type=Unofficial
Error downloading https://www.discogs.com/artist/2062081-Bob-Dylan-With-Tom-Petty-And-The-Heartbreakers?type=Unofficial
Error downloading https://www.discogs.com/artist/227481-DNS-3?type=Unofficial
Error downloading https://www.discogs.com/artist/290581-Big-Joe-Williams?type=Unofficial
Error downloading https://www.discogs.com/artist/334081-Hugo-Rignold?type=Unofficial
Error downloading https://www.discogs.com/artist/3466281-Executive-FourMel-Britt?type=Unofficial
Error downloading https://www.discogs.com/artist/4023281-Roy-Ross-And-His-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/43281-Lio?type=Unofficial
Error downloading https://www.discogs.com/artist/4781-Fellowship?type=Unofficial
Error downloading https://www.discogs.com/artist/58881-Brando?type=Unofficial
Error downloading https://www.discogs.com/artist/6129681-Madonna-Vs-Offer-Nissim?type=Unofficial
Error downloading https://www.discogs.com/artist/688281-Grouper?type=Unofficial
Error downloading https://www.discogs.com/artist/695481-Dingle?type=Unofficial
Error downloading https://www.discogs.com/artist/6981-Gez-Varley?type=Unofficial
Error downloading https://www.discogs.com/artist/80881-KHZ?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/81/unofficial/3664381.p (force=False)
  --> This file is 15.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7733 previous data for ModVal=82
Found 334 artists to get
Error downloading https://www.discogs.com/artist/1085582-Bitter-Funeral-Beer-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/364982-Cliff-Burton?type=Unofficial
=========================   discogs   =========================
  ===> Found 7728 previous data for ModVal=83
Found 337 artists to get
Error downloading https://www.discogs.com/artist/1787083-Dekka-Danse?type=Unofficial
Error downloading https://www.discogs.com/artist/221683-Nitty-2?type=Unofficial
Error downloading https://www.discogs.com/artist/390583-The-Gardenias?type=Unofficial
Error downloading https://www.discogs.com/artist/40083-The-Badman?type=Unofficial
Error downloading https://www.discogs.com/artist/4683983-Hot-Legs2?type=Unofficial
Error downloading https://www.discogs.com/artist/4898783-Wayne-Horvitz-Zony-Mash?type=Unofficial
Error downloading https://www.discogs.com/artist/7683-The-Ultimate-Seduction?type=Unofficial
=========================   discogs   =========================
  ===> Found 7721 previous data for ModVal=84
Found 366 artists to get
Error downloading https://www.discogs.com/artist/1079484-Franchise-3?type=Unofficial
Error downloading https://www.discogs.com/artist/1222784-Ritual-Device?type=Unofficial
Error downloading https://www.discogs.com/artist/157284-Axis-5?type=Unofficial
Error downloading https://www.discogs.com/artist/203684-Ras-Project?type=Unofficial
Error downloading https://www.discogs.com/artist/316984-Funk-Doc?type=Unofficial
Error downloading https://www.discogs.com/artist/455284-June-Carter?type=Unofficial
Error downloading https://www.discogs.com/artist/51084-Pamela-Williams?type=Unofficial
Error downloading https://www.discogs.com/artist/775484-Jim-Ladd?type=Unofficial
Error downloading https://www.discogs.com/artist/801284-Sam-Sparro?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/84/unofficial/789284.p (force=False)
  --> This file is 18.9kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7612 previous data for ModVal=85
Found 323 artists to get
Error downloading https://www.discogs.com/artist/1576685-NDraman-Blintch?type=Unofficial
Error downloading https://www.discogs.com/artist/1987285-Dispensing-Of-False-Halos?type=Unofficial
Error downloading https://www.discogs.com/artist/253885-T-Lee?type=Unofficial
Error downloading https://www.discogs.com/artist/285-Fierce?type=Unofficial
Error downloading https://www.discogs.com/artist/3855185-DJ-Famous?type=Unofficial
Error downloading https://www.discogs.com/artist/597585-Tommy-Brown-2?type=Unofficial
Error downloading https://www.discogs.com/artist/6038385-DJ-Roshi?type=Unofficial
Error downloading https://www.discogs.com/artist/742285-Jay-Gordon-2?type=Unofficial
Error downloading https://www.discogs.com/artist/90185-Kevin-Gilbert?type=Unofficial
=========================   discogs   =========================
  ===> Found 7830 previous data for ModVal=86
Found 340 artists to get
Error downloading https://www.discogs.com/artist/110086-Trigga?type=Unofficial
Error downloading https://www.discogs.com/artist/1484386-Filipe-Rodrigues?type=Unofficial
Error downloading https://www.discogs.com/artist/1612186-Lionel-Jeffries?type=Unofficial
Error downloading https://www.discogs.com/artist/178686-John-Doe-4?type=Unofficial
Error downloading https://www.discogs.com/artist/1791486-Jonathan-Sielaff?type=Unofficial
Error downloading https://www.discogs.com/artist/1996086-Stay-3?type=Unofficial
Error downloading https://www.discogs.com/artist/2296486-The-Archie-Shepp-Bill-Dixon-Quartet?type=Unofficial
Error downloading https://www.discogs.com/artist/272886-David-Gilden?type=Unofficial
Error downloading https://www.discogs.com/artist/282986-Nikki-Belle?type=Unofficial
Error downloading https://www.discogs.com/artist/335786-David-Friesen?type=Unofficial
Error downloading https://www.discogs.com/artist/405286-King-Hannibal?type=Unofficial
Error downloading https://www.discogs.com/artist/4622386-Invasion-17?type=Unofficial
Error downloading https://www.discogs.com/artist/555486-Shines?type=Unofficial
=========================   discogs   =========================
  ===> Found 7650 previous data for ModVal=87
Found 348 artists to get
Error downloading https://www.discogs.com/artist/232887-Friends?type=Unofficial
Error downloading https://www.discogs.com/artist/4487-Katcha?type=Unofficial
Error downloading https://www.discogs.com/artist/6087-Nexus-21?type=Unofficial
Error downloading https://www.discogs.com/artist/71087-Shaka?type=Unofficial
Error downloading https://www.discogs.com/artist/899287-Dave-Alvin-And-The-Guilty-Men?type=Unofficial
=========================   discogs   =========================
  ===> Found 7628 previous data for ModVal=88
Found 347 artists to get
Error downloading https://www.discogs.com/artist/1135188-Black-Mophia-Clan?type=Unofficial
Error downloading https://www.discogs.com/artist/12888-KK-Null?type=Unofficial
Error downloading https://www.discogs.com/artist/2322588-DJ-Ti%C3%91%C2%91sto?type=Unofficial
Error downloading https://www.discogs.com/artist/2793088-Albacore?type=Unofficial
Error downloading https://www.discogs.com/artist/306688-Martin-Simpson?type=Unofficial
Error downloading https://www.discogs.com/artist/3688-Unreleased-Project?type=Unofficial
Error downloading https://www.discogs.com/artist/49288-Shark-Vegas?type=Unofficial
Error downloading https://www.discogs.com/artist/66388-Blue-Nature?type=Unofficial
Error downloading https://www.discogs.com/artist/98688-Billy-Crystal?type=Unofficial
=========================   discogs   =========================
  ===> Found 7742 previous data for ModVal=89
Found 330 artists to get
Error downloading https://www.discogs.com/artist/1796589-Morticite?type=Unofficial
Error downloading https://www.discogs.com/artist/301589-Artie-Butler?type=Unofficial
Error downloading https://www.discogs.com/artist/334389-Rumbleseat?type=Unofficial
Error downloading https://www.discogs.com/artist/3871389-Skilfingar?type=Unofficial
Error downloading https://www.discogs.com/artist/4230689-Kboy-Jack-Dj-s?type=Unofficial
Error downloading https://www.discogs.com/artist/43589-Quartz-2?type=Unofficial
Error downloading https://www.discogs.com/artist/438789-Arrington-De-Dionyso?type=Unofficial
Error downloading https://www.discogs.com/artist/99289-Amit?type=Unofficial
=========================   discogs   =========================
  ===> Found 7528 previous data for ModVal=90
Found 337 artists to get
Error downloading https://www.discogs.com/artist/11490-Industry-Standard-2?type=Unofficial
Error downloading https://www.discogs.com/artist/136990-Aki-Onda?type=Unofficial
Error downloading https://www.discogs.com/artist/1475990-The-John-Berberian-Ensemble?type=Unofficial
Error downloading https://www.discogs.com/artist/171290-Randys-All-Stars?type=Unofficial
Error downloading https://www.discogs.com/artist/2158190-333?type=Unofficial
Error downloading https://www.discogs.com/artist/225490-DJ-Deep-2?type=Unofficial
Error downloading https://www.discogs.com/artist/300790-Brian-Protheroe?type=Unofficial
Error downloading https://www.discogs.com/artist/31590-Bardo-Pond?type=Unofficial
Error downloading https://www.discogs.com/artist/4573790-Suhaily-Shamsuddin?type=Unofficial
=========================   discogs   =========================
  ===> Found 7609 previous data for ModVal=91
Found 340 artists to get
Error downloading https://www.discogs.com/artist/135991-Joe-Coleman?type=Unofficial
Error downloading https://www.discogs.com/artist/1617791-Siam-Shade?type=Unofficial
Error downloading https://www.discogs.com/artist/1629791-Sir-Collins-The-Black-Diamonds?type=Unofficial
Error downloading https://www.discogs.com/artist/17091-Willy-Washington?type=Unofficial
Error downloading https://www.discogs.com/artist/1716591-Les-Prouters?type=Unofficial
Error downloading https://www.discogs.com/artist/191-David-Alvarado?type=Unofficial
Error downloading https://www.discogs.com/artist/2291-Cause-4-Concern?type=Unofficial
Error downloading https://www.discogs.com/artist/38191-Lost?type=Unofficial
Error downloading https://www.discogs.com/artist/405591-Haley-Gibby?type=Unofficial
Error downloading https://www.discogs.com/artist/5891-Pole-Folder-CP?type=Unofficial
Error downloading https://www.discogs.com/artist/624191-Rik-Emmett?type=Unofficial
Error downloading https://www.discogs.com/artist/97191-The-Eternal-Afflict?type=Unofficial
Error downloading https://www.discogs.com/artist/988591-Ted-Joans?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/91/unofficial/543391.p (force=False)
  --> This file is 17.5kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7805 previous data for ModVal=92
Found 320 artists to get
Error downloading https://www.discogs.com/artist/1206992-Dub-Central?type=Unofficial
Error downloading https://www.discogs.com/artist/132592-Phace?type=Unofficial
Error downloading https://www.discogs.com/artist/189392-Orange-Peel?type=Unofficial
Error downloading https://www.discogs.com/artist/194092-Breathe-3?type=Unofficial
Error downloading https://www.discogs.com/artist/212092-De-Kift?type=Unofficial
Error downloading https://www.discogs.com/artist/42192-Mandr%C3%A9?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/92/unofficial/251092.p (force=False)
  --> This file is 15.7kB.
Done. Sleeping for 2 seconds
Saving /Volumes/Piggy/Discog/artists-discogs/92/unofficial/25692.p (force=False)
  --> This file is 17.4kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7627 previous data for ModVal=93
Found 324 artists to get
Error downloading https://www.discogs.com/artist/1013493-Cahill-2?type=Unofficial
Error downloading https://www.discogs.com/artist/216993-Alien-Levi?type=Unofficial
Error downloading https://www.discogs.com/artist/307393-Clarence-Williams?type=Unofficial
Error downloading https://www.discogs.com/artist/385493-The-Quests?type=Unofficial
Error downloading https://www.discogs.com/artist/4167493-Billy-Parker-6?type=Unofficial
Error downloading https://www.discogs.com/artist/450693-Chuck-Berry-His-Combo?type=Unofficial
Error downloading https://www.discogs.com/artist/458093-Fuckpony?type=Unofficial
Error downloading https://www.discogs.com/artist/672493-Fito-P%C3%A1ez?type=Unofficial
Error downloading https://www.discogs.com/artist/993293-Street-Kids?type=Unofficial
=========================   discogs   =========================
  ===> Found 7615 previous data for ModVal=94
Found 351 artists to get
Error downloading https://www.discogs.com/artist/123194-Beth-Gibbons-Rustin-Man?type=Unofficial
Error downloading https://www.discogs.com/artist/21194-DJ-6666-feat-The-Illegals?type=Unofficial
Error downloading https://www.discogs.com/artist/3978394-Maelstrom-26?type=Unofficial
Error downloading https://www.discogs.com/artist/510394-Shinki-Chen?type=Unofficial
Error downloading https://www.discogs.com/artist/5155994-DJB-vs-Who-Da-Fck?type=Unofficial
Error downloading https://www.discogs.com/artist/56194-David-Grubbs?type=Unofficial
=========================   discogs   =========================
  ===> Found 7680 previous data for ModVal=95
Found 319 artists to get
Error downloading https://www.discogs.com/artist/1424095-Isaak?type=Unofficial
Error downloading https://www.discogs.com/artist/259295-DJ-Flapjack?type=Unofficial
Error downloading https://www.discogs.com/artist/29895-Joe-Gibbs-The-Professionals?type=Unofficial
Error downloading https://www.discogs.com/artist/33995-Ed-Motta?type=Unofficial
Error downloading https://www.discogs.com/artist/483695-Boozoo-Chavis?type=Unofficial
Error downloading https://www.discogs.com/artist/494995-Lauren-Newton?type=Unofficial
Error downloading https://www.discogs.com/artist/499495-Brainwork?type=Unofficial
Error downloading https://www.discogs.com/artist/821995-George-Fleming?type=Unofficial
Error downloading https://www.discogs.com/artist/2619095-Jack-Jones-9?type=Unofficial
=========================   discogs   =========================
  ===> Found 7557 previous data for ModVal=96
Found 372 artists to get
Error downloading https://www.discogs.com/artist/1016096-Jeremy-Jay?type=Unofficial
Error downloading https://www.discogs.com/artist/110696-Mummy-Calls?type=Unofficial
Error downloading https://www.discogs.com/artist/124396-DJ-Extreme?type=Unofficial
Error downloading https://www.discogs.com/artist/2131996-KDC?type=Unofficial
Error downloading https://www.discogs.com/artist/255596-Mystic-Tide?type=Unofficial
Error downloading https://www.discogs.com/artist/28596-Nero?type=Unofficial
Error downloading https://www.discogs.com/artist/3396496-Temple-Of-Oblivion?type=Unofficial
Error downloading https://www.discogs.com/artist/3628996-The-Present-5?type=Unofficial
Error downloading https://www.discogs.com/artist/3696-Nucleus-Paradox?type=Unofficial
Error downloading https://www.discogs.com/artist/83696-Frighty?type=Unofficial
Error downloading https://www.discogs.com/artist/840496-The-Dub-Beatles?type=Unofficial
Error downloading https://www.discogs.com/artist/8996-TD?type=Unofficial
=========================   discogs   =========================
  ===> Found 7619 previous data for ModVal=97
Found 322 artists to get
Error downloading https://www.discogs.com/artist/2190897-Konwent?type=Unofficial
Error downloading https://www.discogs.com/artist/2625697-J-Bigga?type=Unofficial
Error downloading https://www.discogs.com/artist/268097-Billy-Nicholls?type=Unofficial
Error downloading https://www.discogs.com/artist/30897-Timex-Social-Club?type=Unofficial
Error downloading https://www.discogs.com/artist/479897-Sonny-Moore?type=Unofficial
=========================   discogs   =========================
  ===> Found 7563 previous data for ModVal=98
Found 321 artists to get
Error downloading https://www.discogs.com/artist/1747298-Dilermando-Reis?type=Unofficial
Error downloading https://www.discogs.com/artist/220598-Q-ic?type=Unofficial
Error downloading https://www.discogs.com/artist/22898-Tool?type=Unofficial
=========================   discogs   =========================
  ===> Found 7646 previous data for ModVal=99
Found 325 artists to get
Error downloading https://www.discogs.com/artist/10199-Diesel?type=Unofficial
Error downloading https://www.discogs.com/artist/123599-Ram-Jam-World?type=Unofficial
Error downloading https://www.discogs.com/artist/19099-Necro?type=Unofficial
Error downloading https://www.discogs.com/artist/193299-Deon-Estus?type=Unofficial
Error downloading https://www.discogs.com/artist/21699-3-Way?type=Unofficial
Error downloading https://www.discogs.com/artist/2430899-Darren-Korb?type=Unofficial
Error downloading https://www.discogs.com/artist/299-Rufige-Kru?type=Unofficial
Error downloading https://www.discogs.com/artist/329799-Zbigniew-Seifert?type=Unofficial
Error downloading https://www.discogs.com/artist/4987899-Cross-Quinn?type=Unofficial
Error downloading https://www.discogs.com/artist/5217299-Ural-Thomas-And-The-Pain?type=Unofficial
Error downloading https://www.discogs.com/artist/87799-Severo-Lombardoni?type=Unofficial"""

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

dbAD = dbArtistsDiscogs()
dbAP = dbArtistsPrimary(dbAD)

removes = []
for i,line in enumerate(vals.split("\n")):
    if i < 2:
        continue
    if not line.startswith("Error downloading "):
        continue
    url = line.replace("Error downloading ", "")
    
    print(url)
    
    url = url.replace("?type=Unofficial", "")
    artistID = dbAP.dutils.getArtistID(url, debug=False)
    savename = dbAP.dutils.getArtistSavename(artistID)
    print(url)

    try:
        dbAP.dutils.downloadArtistURL(url, savename, force=True, sleeptime=2)
    except:
        removes.append(line)
        

In [ ]:
dbAP.dutils.getArtistID("https://www.discogs.com/artist/718900-G-Punkt-2")

In [ ]:

artistURL = "https://www.discogs.com/artist/2428401-Horse-Head-2?type=Unofficial"
print(artistID,savename)